In [1]:
!pip install -q sentence-transformers

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
with open("/content/SBERT.json", "r") as f:
    triplet_data = json.load(f)

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from torch.utils.data import DataLoader
import pandas as pd

train_examples = []
for item in triplet_data:
    anchor = item['anchor']
    for positive in item['positives']:
      print(positive)
      for negative in item['negatives']:
          print(negative)
          train_examples.append(InputExample(texts=[anchor, positive, negative]))

In [ ]:
model_name = "hyrinmansoor/text2frappe-s2-sbert"
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [ ]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

In [ ]:
train_loss = losses.TripletLoss(model=model, distance_metric=losses.TripletDistanceMetric.COSINE, triplet_margin=0.3)

In [ ]:
num_epochs =15

In [ ]:
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)

In [ ]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    output_path="/content/drive/MyDrive/Changai/S2/Model/Test1"
)

In [ ]:
model.save("/content/drive/MyDrive/Changai/S2/Model/Test1")

In [ ]:
import json
from google.colab import drive
with open("/content/meta.json") as f:
  meta=json.load(f)

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer("/content/drive/MyDrive/Changai/S2/S2 Model/sbert_13_08_25_6")

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_folder(
    folder_path="/content/drive/MyDrive/Changai/S2/Model/Test1",
    repo_id="hyrinmansoor/text2frappe-s2-sbert",
    repo_type="model"
)

In [ ]:
from sentence_transformers import SentenceTransformer, util
import re

model = SentenceTransformer("hyrinmansoor/text2frappe-s2-sbert")
expected_fields_map = [
    {
        "anchor": "Doctype: Sales Order\nQuestion: List all sales orders scheduled for delivery this week.",
        "expected_fields": ["name", "delivery_date", "customer"]
    },
    {
        "anchor": "Doctype: Sales Order\nQuestion: Show sales orders with a discount greater than 10%.",
        "expected_fields": ["name", "discount_amount", "customer"]
    },
    {
        "anchor": "Doctype: Sales Order\nQuestion: List all sales orders for project 'Sunrise Apartments'.",
        "expected_fields": ["name", "project", "customer"]
    },
    {
        "anchor": "Doctype: Purchase Order\nQuestion: Show purchase orders pending approval.",
        "expected_fields": ["name", "docstatus", "supplier"]
    },
    {
        "anchor": "Doctype: Purchase Order\nQuestion: List purchase orders with expected delivery this month.",
        "expected_fields": ["name", "expected_delivery_date", "supplier"]
    },
    {
        "anchor": "Doctype: Customer\nQuestion: Show customers with credit limit above 100,000.",
        "expected_fields": ["name", "customer_name", "credit_limit"]
    },
    {
        "anchor": "Doctype: Customer\nQuestion: List all inactive customers.",
        "expected_fields": ["name", "customer_name", "status"]
    },
    {
        "anchor": "Doctype: Employee\nQuestion: Show employees in the Marketing department.",
        "expected_fields": ["name", "employee_name", "department"]
    },
    {
        "anchor": "Doctype: Employee\nQuestion: List employees whose contract ends before 31-Dec-2025.",
        "expected_fields": ["name", "employee_name", "contract_end_date"]
    },
    {
        "anchor": "Doctype: Item\nQuestion: Show items with reorder level below 20.",
        "expected_fields": ["item_code", "item_name", "reorder_level", "actual_qty"]
    },
    {
        "anchor": "Doctype: Item\nQuestion: List items with last purchase rate above 2,000.",
        "expected_fields": ["item_code", "item_name", "last_purchase_rate"]
    },
    {
        "anchor": "Doctype: Sales Order\nQuestion: List all sales orders scheduled for delivery this week.",
        "expected_fields": ["name", "delivery_date", "customer"]
    },
    {
        "anchor": "Doctype: Sales Order\nQuestion: Show sales orders with a discount greater than 10%.",
        "expected_fields": ["name", "discount_amount", "customer"]
    },
    {
        "anchor": "Doctype: Sales Order\nQuestion: List all sales orders for project 'Sunrise Apartments'.",
        "expected_fields": ["name", "project", "customer"]
    },
    {
        "anchor": "Doctype: Sales Order\nQuestion: Show pending sales orders not yet delivered.",
        "expected_fields": ["name", "delivery_date", "customer"]
    },
    {
        "anchor": "Doctype: Sales Order\nQuestion: List sales orders with grand total above 100,000 for VIP customers.",
        "expected_fields": ["name", "grand_total", "customer"]
    },
        {
        "anchor": "Doctype: Purchase Order\nQuestion: Show purchase orders pending approval.",
        "expected_fields": ["name", "docstatus", "supplier"]
    },
    {
        "anchor": "Doctype: Purchase Order\nQuestion: List purchase orders with expected delivery this month.",
        "expected_fields": ["name", "expected_delivery_date", "supplier"]
    },
    {
        "anchor": "Doctype: Purchase Order\nQuestion: Show purchase orders with total amount above 500,000.",
        "expected_fields": ["name", "grand_total", "supplier"]
    },
    {
        "anchor": "Doctype: Purchase Order\nQuestion: List purchase orders received from supplier 'ABC Pvt Ltd'.",
        "expected_fields": ["name", "supplier", "transaction_date"]
    },
        {
        "anchor": "Doctype: Customer\nQuestion: Show customers with credit limit above 100,000.",
        "expected_fields": ["name", "customer_name", "credit_limit"]
    },
    {
        "anchor": "Doctype: Customer\nQuestion: List all inactive customers.",
        "expected_fields": ["name", "customer_name", "status"]
    },
    {
        "anchor": "Doctype: Customer\nQuestion: List customers in the 'Retail' market segment.",
        "expected_fields": ["name", "customer_name", "market_segment"]
    },
    {
        "anchor": "Doctype: Customer\nQuestion: Show customers with outstanding invoices above 50,000.",
        "expected_fields": ["name", "customer_name", "outstanding_amount"]
    },
    {
        "anchor": "Doctype: Customer\nQuestion: Show VIP customers in California with credit limit above 200,000.",
        "expected_fields": ["name", "customer_name", "credit_limit"]
    },
        {
        "anchor": "Doctype: Employee\nQuestion: Show employees in the Marketing department.",
        "expected_fields": ["name", "employee_name", "department"]
    },
    {
        "anchor": "Doctype: Employee\nQuestion: List employees whose contract ends before 31-Dec-2025.",
        "expected_fields": ["name", "employee_name", "contract_end_date"]
    },
    {
        "anchor": "Doctype: Employee\nQuestion: Show employees with salary above 50,000.",
        "expected_fields": ["name", "employee_name", "ctc"]
    },
    {
        "anchor": "Doctype: Employee\nQuestion: List employees joined after 01-Jan-2024 in Sales department.",
        "expected_fields": ["name", "employee_name", "date_of_joining", "department"]
    },
    {
        "anchor": "Doctype: Employee\nQuestion: List employees on contract expiring this year with active status.",
        "expected_fields": ["name", "employee_name", "contract_end_date", "status"]
    },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: How many sales invoices were created last month?",
    "expected_fields": ["name", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: What is the total outstanding amount for all sales invoices?",
    "expected_fields": ["name", "outstanding_amount"]
  },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: List all sales invoices with their name and status.",
    "expected_fields": ["name", "status"]
  },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: What is the status of sales invoice SI-0001?",
    "expected_fields": ["name", "status"]
  },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: How many sales returns were recorded this quarter?",
    "expected_fields": ["name", "is_return", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: What is the total revenue from sales invoices this year?",
    "expected_fields": ["name", "grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: Which customer has the highest total invoice amount this year?",
    "expected_fields": ["customer", "grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: How many invoices were submitted last week?",
    "expected_fields": ["name", "posting_date", "docstatus"]
  },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: What is the average invoice amount this month?",
    "expected_fields": ["grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: List all invoices created today.",
    "expected_fields": ["name", "posting_date"]
  },

  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: How many purchase invoices were created this month?",
    "expected_fields": ["name", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: What is the total outstanding amount for all purchase invoices?",
    "expected_fields": ["name", "outstanding_amount"]
  },
  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: List all purchase invoices with their supplier name and status.",
    "expected_fields": ["name", "supplier", "status"]
  },
  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: What is the status of purchase invoice PI-0005?",
    "expected_fields": ["name", "status"]
  },
  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: How many purchase returns were recorded last quarter?",
    "expected_fields": ["name", "is_return", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: What is the total expenditure from purchase invoices this year?",
    "expected_fields": ["name", "grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: Which supplier has the highest total purchase invoice amount?",
    "expected_fields": ["supplier", "grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: How many purchase invoices were submitted last week?",
    "expected_fields": ["name", "posting_date", "docstatus"]
  },
  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: What is the average purchase invoice amount this month?",
    "expected_fields": ["grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: List all purchase invoices created today.",
    "expected_fields": ["name", "posting_date"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: How many employees are currently active?",
    "expected_fields": ["name", "status"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: List all employees with their department and designation.",
    "expected_fields": ["name", "department", "designation"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: What is the date of joining for employee EMP-0001?",
    "expected_fields": ["name", "date_of_joining"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: Which employee has the highest salary?",
    "expected_fields": ["name", "salary"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: How many employees joined in the last quarter?",
    "expected_fields": ["name", "date_of_joining"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: List all employees whose contract ends this month.",
    "expected_fields": ["name", "contract_end_date"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: What is the average salary in the company?",
    "expected_fields": ["salary"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: Which department has the highest average salary?",
    "expected_fields": ["department", "salary"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: How many employees are on probation?",
    "expected_fields": ["name", "employment_type"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: List employees who have pending performance reviews.",
    "expected_fields": ["name", "performance_review_status"]
  },

  {
    "anchor": "Doctype: Sales Order\nQuestion: How many sales orders were created this month?",
    "expected_fields": ["name", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Order\nQuestion: What is the total amount for all sales orders this year?",
    "expected_fields": ["name", "grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Order\nQuestion: List all sales orders with their customer and status.",
    "expected_fields": ["name", "customer", "status"]
  },
  {
    "anchor": "Doctype: Sales Order\nQuestion: What is the status of sales order SO-0003?",
    "expected_fields": ["name", "status"]
  },
  {
    "anchor": "Doctype: Sales Order\nQuestion: How many sales orders were delivered last month?",
    "expected_fields": ["name", "delivery_date", "status"]
  },
  {
    "anchor": "Doctype: Sales Order\nQuestion: Which customer has the highest total sales order value?",
    "expected_fields": ["customer", "grand_total"]
  },
  {
    "anchor": "Doctype: Sales Order\nQuestion: How many sales orders were cancelled last quarter?",
    "expected_fields": ["name", "status", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Order\nQuestion: What is the average sales order amount this year?",
    "expected_fields": ["grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Order\nQuestion: List all sales orders created today.",
    "expected_fields": ["name", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Order\nQuestion: How many sales orders are pending delivery?",
    "expected_fields": ["name", "status"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: How many purchase orders were created last month?",
    "expected_fields": ["name", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: What is the total value of all purchase orders this year?",
    "expected_fields": ["name", "grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: List all purchase orders with their supplier and status.",
    "expected_fields": ["name", "supplier", "status"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: What is the status of purchase order PO-0007?",
    "expected_fields": ["name", "status"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: How many purchase orders were received in the last quarter?",
    "expected_fields": ["name", "schedule_date", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: Which supplier has the highest total purchase order value?",
    "expected_fields": ["supplier", "grand_total"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: How many purchase orders were cancelled last month?",
    "expected_fields": ["name", "status", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: What is the average purchase order value this year?",
    "expected_fields": ["grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: List all purchase orders created today.",
    "expected_fields": ["name", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: How many purchase orders are pending receipt?",
    "expected_fields": ["name", "status"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: How many items are currently in stock?",
    "expected_fields": ["item_code", "actual_qty"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: List all items with their item code and stock quantity.",
    "expected_fields": ["item_code", "item_name", "actual_qty"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: What is the stock quantity of item ITEM-0001?",
    "expected_fields": ["item_code", "actual_qty"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: Which item has the highest stock quantity?",
    "expected_fields": ["item_code", "item_name", "actual_qty"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: How many items are below the reorder level?",
    "expected_fields": ["item_code", "reorder_level", "actual_qty"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: List all items that were added this month.",
    "expected_fields": ["item_code", "item_name", "creation"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: What is the average selling price of items?",
    "expected_fields": ["item_code", "standard_rate"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: Which item has the highest selling price?",
    "expected_fields": ["item_code", "item_name", "standard_rate"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: How many items have no stock?",
    "expected_fields": ["item_code", "actual_qty"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: List all discontinued items.",
    "expected_fields": ["item_code", "item_name", "disabled"]
  },
    {
        "anchor": "Doctype: Item\nQuestion: Show items with reorder level below 20.",
        "expected_fields": ["item_code", "item_name", "reorder_level", "actual_qty"]
    },
    {
        "anchor": "Doctype: Item\nQuestion: List items with last purchase rate above 2,000.",
        "expected_fields": ["item_code", "item_name", "last_purchase_rate"]
    },
    {
        "anchor": "Doctype: Item\nQuestion: Show items with stock below 50 units in warehouse 'Main'.",
        "expected_fields": ["item_code", "item_name", "actual_qty", "stock_uom"]
    },
    {
        "anchor": "Doctype: Item\nQuestion: List items with standard rate above 5,000 and active status.",
        "expected_fields": ["item_code", "item_name", "standard_rate", "disabled"]
    },
    {
        "anchor": "Doctype: Item\nQuestion: Show items supplied by 'XYZ Supplier' with last purchase rate above 1,000.",
        "expected_fields": ["item_code", "item_name", "last_purchase_rate", "supplier_items"]
    }
]
from sentence_transformers import SentenceTransformer, util
import re
for item in expected_fields_map:
    anchor = item["anchor"]
    expected = item["expected_fields"]

    match = re.match(r"Doctype:\s*(.*?)\s*\nQuestion:\s*(.*)", anchor)
    if not match:
        print(f"Invalid format: {anchor}")
        continue

    doctype = match.group(1)
    question = match.group(2)

    print(f"\nDoctype: {doctype}")
    print(f"Question: {question}")

    # Get all fields for this doctype
    candidate_fields = meta.get(doctype, {}).get("fields", [])
    if not candidate_fields:
        print(f"No fields found for doctype '{doctype}'. Skipping...")
        continue

    # Encode question and fields
    question_emb = model.encode(question, convert_to_tensor=True)
    field_embs = model.encode(candidate_fields, convert_to_tensor=True)

    # Rank fields by similarity
    scores = util.cos_sim(question_emb, field_embs)[0]
    ranked = sorted(zip(candidate_fields, scores), key=lambda x: x[1], reverse=True)

    print("Top ranked fields:")
    for field, score in ranked[:5]:
        print(f"  {field} -> {score:.4f}")

    # Compare with expected fields
    top_fields = [f for f, _ in ranked[:5]]
    hits = [f for f in expected if f in top_fields]
    print(f"Expected fields: {expected}")
    print(f"Matches in top 5: {hits}")

In [ ]:
model.fit(train_objectives=[(train_dataloader,train_Loss)],
          epochs=3,
          warmup_steps=32,
          output_path=output_path
          )

In [ ]:
def extract_doctype(text):
    for line in text.split("\n"):
        if line.lower().startswith("doctype:"):
            return line.split(":", 1)[1].strip()
    return None

In [ ]:
def get_fields_for_doctype(meta: dict, doctype: str):
    if doctype not in meta:
        raise ValueError(f"❌ Doctype '{doctype}' not found in metadata.")
    fields_dict = meta[doctype]["fields"]
    return [f"{key}: {desc}" for key, desc in fields_dict.items()]

In [ ]:
with open("/content/drive/MyDrive/Changai/meta.json") as f:
  meta=json.load(f)

In [ ]:
meta

In [ ]:
from huggingface_hub import login

login()

In [ ]:
from huggingface_hub import create_repo

create_repo("text2frappe-s2-sbert", private=True)

In [ ]:
from huggingface_hub import upload_folder

upload_folder(
    repo_id="hyrinmansoor/text2frappe-s2-sbert",
    folder_path="/content/drive/MyDrive/Changai/S2/S2 Model/sbert_topfield_selector",
    path_in_repo=".",  # root of the model repo
    repo_type="model"
)


### 🔗 Load Fine-Tuned SBERT Model from Hugging Face Hub (Stage 2 - Doctype Classification)


In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

model_name = "hyrinmansoor/text2frappe-s2-sbert"  # can be swapped anytime
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Now you can call through API Inference also.
#API_URL = 'https://huggingface.co/hyrinmansoor/text2frappe-s2-sbert'

### SBERT-Based Top Field Selector Evaluation (Stage 2 - Field Ranking by Semantic Similarity)



In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

def extract_doctype(query):
    for line in query.split("\n"):
        if line.lower().startswith("doctype:"):
            return line.split(":", 1)[1].strip()
    return None

def get_fields_for_doctype(meta, doctype):
    if doctype not in meta:
        raise ValueError(f"❌ Doctype '{doctype}' not found in metadata.")
    fields_data = meta[doctype].get("fields")
    return fields_data
test_cases=[
    (
        "Doctype: Purchase Invoice Advance\nQuestion: How much of the advance was allocated for PINV-00942?",
        ["allocated_amount"]
    ),
    (
        "Doctype: Purchase Invoice Advance\nQuestion: What exchange rate was applied to document PINV-0452?",
        ["ref_exchange_rate"]
    ),
    (
        "Doctype: Closing Stock Balance\nQuestion: Check whether the stock balance entry created on '2024-04-01' is still in Draft.",
        ["status"]
    ),
    (
        "Doctype: Sales Invoice Advance\nQuestion: Find the exchange loss on any entries with ref exchange rate below 3.5.",
        ["exchange_gain_loss"]
    ),
    (
        "Doctype: Sales Invoice Advance\nQuestion: What portion of advance has been allocated in entry SINVADV-2217?",
        ["allocated_amount"]
    )
]

model = SentenceTransformer("/content/drive/MyDrive/Changai/S2/S2 Model/sbert_topfield_selector")

for i, (query, expected_fields) in enumerate(test_cases):
    doctype = extract_doctype(query)
    print(doctype)
    if doctype is None:
        print(f"\nTest Case {i+1}: Skipping - Could not extract doctype from query.")
        continue

    try:
        fields = get_fields_for_doctype(meta, doctype)
    except ValueError as e:
        print(f"\nTest Case {i+1}: Skipping - {e}")
        continue

    query_embedding = model.encode(query, convert_to_tensor=True)
    field_embeddings = model.encode(fields, convert_to_tensor=True)
    scores = util.cos_sim(query_embedding, field_embeddings)
    k = min(len(fields), 5)
    top_k = torch.topk(scores, k=len(fields))

    print(f"\nTest Case {i+1}")
    print("="*60)
    print(f"Query: {query}\n")
    print(f"Extracted Doctype: {doctype}")
    print("Ranked Fields (Top 5):")

    for idx, score in zip(top_k.indices[0][:5], top_k.values[0][:5]):
        field_name = fields[idx]
        mark = "✅" if field_name in expected_fields else "❌"
        print(f"{mark} {field_name} (score: {score:.4f})")

    top5_fields = [fields[idx] for idx in top_k.indices[0][:5]]
    missed = [f for f in expected_fields if f not in top5_fields]
    if missed:
        print("❗Expected fields NOT in top 5:", missed)
    else:
        print("🎉 All expected fields are in the top 5!\n")

In [ ]:
from sentence_transformers import SentenceTransformer, util
import re

model = SentenceTransformer("hyrinmansoor/text2frappe-s2-sbert")
expected_fields_map = [
    {
        "anchor": "Doctype: Sales Order\nQuestion: List all sales orders scheduled for delivery this week.",
        "expected_fields": ["name", "delivery_date", "customer"]
    },
    {
        "anchor": "Doctype: Sales Order\nQuestion: Show sales orders with a discount greater than 10%.",
        "expected_fields": ["name", "discount_amount", "customer"]
    },
    {
        "anchor": "Doctype: Sales Order\nQuestion: List all sales orders for project 'Sunrise Apartments'.",
        "expected_fields": ["name", "project", "customer"]
    },
    {
        "anchor": "Doctype: Purchase Order\nQuestion: Show purchase orders pending approval.",
        "expected_fields": ["name", "docstatus", "supplier"]
    },
    {
        "anchor": "Doctype: Purchase Order\nQuestion: List purchase orders with expected delivery this month.",
        "expected_fields": ["name", "expected_delivery_date", "supplier"]
    },
    {
        "anchor": "Doctype: Customer\nQuestion: Show customers with credit limit above 100,000.",
        "expected_fields": ["name", "customer_name", "credit_limit"]
    },
    {
        "anchor": "Doctype: Customer\nQuestion: List all inactive customers.",
        "expected_fields": ["name", "customer_name", "status"]
    },
    {
        "anchor": "Doctype: Employee\nQuestion: Show employees in the Marketing department.",
        "expected_fields": ["name", "employee_name", "department"]
    },
    {
        "anchor": "Doctype: Employee\nQuestion: List employees whose contract ends before 31-Dec-2025.",
        "expected_fields": ["name", "employee_name", "contract_end_date"]
    },
    {
        "anchor": "Doctype: Item\nQuestion: Show items with reorder level below 20.",
        "expected_fields": ["item_code", "item_name", "reorder_level", "actual_qty"]
    },
    {
        "anchor": "Doctype: Item\nQuestion: List items with last purchase rate above 2,000.",
        "expected_fields": ["item_code", "item_name", "last_purchase_rate"]
    },
    {
        "anchor": "Doctype: Sales Order\nQuestion: List all sales orders scheduled for delivery this week.",
        "expected_fields": ["name", "delivery_date", "customer"]
    },
    {
        "anchor": "Doctype: Sales Order\nQuestion: Show sales orders with a discount greater than 10%.",
        "expected_fields": ["name", "discount_amount", "customer"]
    },
    {
        "anchor": "Doctype: Sales Order\nQuestion: List all sales orders for project 'Sunrise Apartments'.",
        "expected_fields": ["name", "project", "customer"]
    },
    {
        "anchor": "Doctype: Sales Order\nQuestion: Show pending sales orders not yet delivered.",
        "expected_fields": ["name", "delivery_date", "customer"]
    },
    {
        "anchor": "Doctype: Sales Order\nQuestion: List sales orders with grand total above 100,000 for VIP customers.",
        "expected_fields": ["name", "grand_total", "customer"]
    },
        {
        "anchor": "Doctype: Purchase Order\nQuestion: Show purchase orders pending approval.",
        "expected_fields": ["name", "docstatus", "supplier"]
    },
    {
        "anchor": "Doctype: Purchase Order\nQuestion: List purchase orders with expected delivery this month.",
        "expected_fields": ["name", "expected_delivery_date", "supplier"]
    },
    {
        "anchor": "Doctype: Purchase Order\nQuestion: Show purchase orders with total amount above 500,000.",
        "expected_fields": ["name", "grand_total", "supplier"]
    },
    {
        "anchor": "Doctype: Purchase Order\nQuestion: List purchase orders received from supplier 'ABC Pvt Ltd'.",
        "expected_fields": ["name", "supplier", "transaction_date"]
    },
        {
        "anchor": "Doctype: Customer\nQuestion: Show customers with credit limit above 100,000.",
        "expected_fields": ["name", "customer_name", "credit_limit"]
    },
    {
        "anchor": "Doctype: Customer\nQuestion: List all inactive customers.",
        "expected_fields": ["name", "customer_name", "status"]
    },
    {
        "anchor": "Doctype: Customer\nQuestion: List customers in the 'Retail' market segment.",
        "expected_fields": ["name", "customer_name", "market_segment"]
    },
    {
        "anchor": "Doctype: Customer\nQuestion: Show customers with outstanding invoices above 50,000.",
        "expected_fields": ["name", "customer_name", "outstanding_amount"]
    },
    {
        "anchor": "Doctype: Customer\nQuestion: Show VIP customers in California with credit limit above 200,000.",
        "expected_fields": ["name", "customer_name", "credit_limit"]
    },
        {
        "anchor": "Doctype: Employee\nQuestion: Show employees in the Marketing department.",
        "expected_fields": ["name", "employee_name", "department"]
    },
    {
        "anchor": "Doctype: Employee\nQuestion: List employees whose contract ends before 31-Dec-2025.",
        "expected_fields": ["name", "employee_name", "contract_end_date"]
    },
    {
        "anchor": "Doctype: Employee\nQuestion: Show employees with salary above 50,000.",
        "expected_fields": ["name", "employee_name", "ctc"]
    },
    {
        "anchor": "Doctype: Employee\nQuestion: List employees joined after 01-Jan-2024 in Sales department.",
        "expected_fields": ["name", "employee_name", "date_of_joining", "department"]
    },
    {
        "anchor": "Doctype: Employee\nQuestion: List employees on contract expiring this year with active status.",
        "expected_fields": ["name", "employee_name", "contract_end_date", "status"]
    },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: How many sales invoices were created last month?",
    "expected_fields": ["name", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: What is the total outstanding amount for all sales invoices?",
    "expected_fields": ["name", "outstanding_amount"]
  },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: List all sales invoices with their name and status.",
    "expected_fields": ["name", "status"]
  },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: What is the status of sales invoice SI-0001?",
    "expected_fields": ["name", "status"]
  },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: How many sales returns were recorded this quarter?",
    "expected_fields": ["name", "is_return", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: What is the total revenue from sales invoices this year?",
    "expected_fields": ["name", "grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: Which customer has the highest total invoice amount this year?",
    "expected_fields": ["customer", "grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: How many invoices were submitted last week?",
    "expected_fields": ["name", "posting_date", "docstatus"]
  },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: What is the average invoice amount this month?",
    "expected_fields": ["grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Invoice\nQuestion: List all invoices created today.",
    "expected_fields": ["name", "posting_date"]
  },

  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: How many purchase invoices were created this month?",
    "expected_fields": ["name", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: What is the total outstanding amount for all purchase invoices?",
    "expected_fields": ["name", "outstanding_amount"]
  },
  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: List all purchase invoices with their supplier name and status.",
    "expected_fields": ["name", "supplier", "status"]
  },
  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: What is the status of purchase invoice PI-0005?",
    "expected_fields": ["name", "status"]
  },
  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: How many purchase returns were recorded last quarter?",
    "expected_fields": ["name", "is_return", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: What is the total expenditure from purchase invoices this year?",
    "expected_fields": ["name", "grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: Which supplier has the highest total purchase invoice amount?",
    "expected_fields": ["supplier", "grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: How many purchase invoices were submitted last week?",
    "expected_fields": ["name", "posting_date", "docstatus"]
  },
  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: What is the average purchase invoice amount this month?",
    "expected_fields": ["grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Invoice\nQuestion: List all purchase invoices created today.",
    "expected_fields": ["name", "posting_date"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: How many employees are currently active?",
    "expected_fields": ["name", "status"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: List all employees with their department and designation.",
    "expected_fields": ["name", "department", "designation"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: What is the date of joining for employee EMP-0001?",
    "expected_fields": ["name", "date_of_joining"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: Which employee has the highest salary?",
    "expected_fields": ["name", "salary"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: How many employees joined in the last quarter?",
    "expected_fields": ["name", "date_of_joining"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: List all employees whose contract ends this month.",
    "expected_fields": ["name", "contract_end_date"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: What is the average salary in the company?",
    "expected_fields": ["salary"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: Which department has the highest average salary?",
    "expected_fields": ["department", "salary"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: How many employees are on probation?",
    "expected_fields": ["name", "employment_type"]
  },
  {
    "anchor": "Doctype: Employee\nQuestion: List employees who have pending performance reviews.",
    "expected_fields": ["name", "performance_review_status"]
  },

  {
    "anchor": "Doctype: Sales Order\nQuestion: How many sales orders were created this month?",
    "expected_fields": ["name", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Order\nQuestion: What is the total amount for all sales orders this year?",
    "expected_fields": ["name", "grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Order\nQuestion: List all sales orders with their customer and status.",
    "expected_fields": ["name", "customer", "status"]
  },
  {
    "anchor": "Doctype: Sales Order\nQuestion: What is the status of sales order SO-0003?",
    "expected_fields": ["name", "status"]
  },
  {
    "anchor": "Doctype: Sales Order\nQuestion: How many sales orders were delivered last month?",
    "expected_fields": ["name", "delivery_date", "status"]
  },
  {
    "anchor": "Doctype: Sales Order\nQuestion: Which customer has the highest total sales order value?",
    "expected_fields": ["customer", "grand_total"]
  },
  {
    "anchor": "Doctype: Sales Order\nQuestion: How many sales orders were cancelled last quarter?",
    "expected_fields": ["name", "status", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Order\nQuestion: What is the average sales order amount this year?",
    "expected_fields": ["grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Order\nQuestion: List all sales orders created today.",
    "expected_fields": ["name", "posting_date"]
  },
  {
    "anchor": "Doctype: Sales Order\nQuestion: How many sales orders are pending delivery?",
    "expected_fields": ["name", "status"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: How many purchase orders were created last month?",
    "expected_fields": ["name", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: What is the total value of all purchase orders this year?",
    "expected_fields": ["name", "grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: List all purchase orders with their supplier and status.",
    "expected_fields": ["name", "supplier", "status"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: What is the status of purchase order PO-0007?",
    "expected_fields": ["name", "status"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: How many purchase orders were received in the last quarter?",
    "expected_fields": ["name", "schedule_date", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: Which supplier has the highest total purchase order value?",
    "expected_fields": ["supplier", "grand_total"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: How many purchase orders were cancelled last month?",
    "expected_fields": ["name", "status", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: What is the average purchase order value this year?",
    "expected_fields": ["grand_total", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: List all purchase orders created today.",
    "expected_fields": ["name", "posting_date"]
  },
  {
    "anchor": "Doctype: Purchase Order\nQuestion: How many purchase orders are pending receipt?",
    "expected_fields": ["name", "status"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: How many items are currently in stock?",
    "expected_fields": ["item_code", "actual_qty"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: List all items with their item code and stock quantity.",
    "expected_fields": ["item_code", "item_name", "actual_qty"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: What is the stock quantity of item ITEM-0001?",
    "expected_fields": ["item_code", "actual_qty"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: Which item has the highest stock quantity?",
    "expected_fields": ["item_code", "item_name", "actual_qty"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: How many items are below the reorder level?",
    "expected_fields": ["item_code", "reorder_level", "actual_qty"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: List all items that were added this month.",
    "expected_fields": ["item_code", "item_name", "creation"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: What is the average selling price of items?",
    "expected_fields": ["item_code", "standard_rate"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: Which item has the highest selling price?",
    "expected_fields": ["item_code", "item_name", "standard_rate"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: How many items have no stock?",
    "expected_fields": ["item_code", "actual_qty"]
  },
  {
    "anchor": "Doctype: Item\nQuestion: List all discontinued items.",
    "expected_fields": ["item_code", "item_name", "disabled"]
  },
    {
        "anchor": "Doctype: Item\nQuestion: Show items with reorder level below 20.",
        "expected_fields": ["item_code", "item_name", "reorder_level", "actual_qty"]
    },
    {
        "anchor": "Doctype: Item\nQuestion: List items with last purchase rate above 2,000.",
        "expected_fields": ["item_code", "item_name", "last_purchase_rate"]
    },
    {
        "anchor": "Doctype: Item\nQuestion: Show items with stock below 50 units in warehouse 'Main'.",
        "expected_fields": ["item_code", "item_name", "actual_qty", "stock_uom"]
    },
    {
        "anchor": "Doctype: Item\nQuestion: List items with standard rate above 5,000 and active status.",
        "expected_fields": ["item_code", "item_name", "standard_rate", "disabled"]
    },
    {
        "anchor": "Doctype: Item\nQuestion: Show items supplied by 'XYZ Supplier' with last purchase rate above 1,000.",
        "expected_fields": ["item_code", "item_name", "last_purchase_rate", "supplier_items"]
    }
]
from sentence_transformers import SentenceTransformer, util
import re
for item in expected_fields_map:
    anchor = item["anchor"]
    expected = item["expected_fields"]

    match = re.match(r"Doctype:\s*(.*?)\s*\nQuestion:\s*(.*)", anchor)
    if not match:
        print(f"Invalid format: {anchor}")
        continue

    doctype = match.group(1)
    question = match.group(2)

    print(f"\nDoctype: {doctype}")
    print(f"Question: {question}")

    # Get all fields for this doctype
    candidate_fields = meta.get(doctype, {}).get("fields", [])
    if not candidate_fields:
        print(f"No fields found for doctype '{doctype}'. Skipping...")
        continue

    # Encode question and fields
    question_emb = model.encode(question, convert_to_tensor=True)
    field_embs = model.encode(candidate_fields, convert_to_tensor=True)

    # Rank fields by similarity
    scores = util.cos_sim(question_emb, field_embs)[0]
    ranked = sorted(zip(candidate_fields, scores), key=lambda x: x[1], reverse=True)

    print("Top ranked fields:")
    for field, score in ranked[:5]:
        print(f"  {field} -> {score:.4f}")

    # Compare with expected fields
    top_fields = [f for f, _ in ranked[:5]]
    hits = [f for f in expected if f in top_fields]
    print(f"Expected fields: {expected}")
    print(f"Matches in top 5: {hits}")



In [ ]:
from huggingface_hub import upload_folder

upload_folder(
    folder_path="/content/drive/MyDrive/Changai/S2/S2 Model/SBERT- 11-08-2025 2",
    repo_id="hyrinmansoor/text2frappe-s2-sbert",
    repo_type="model"  # or "dataset"
)


In [ ]:
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.util import cos_sim
doctype_meta = {dt: fields_info["fields"] for dt, fields_info in meta.items()}
model = SentenceTransformer("hyrinmansoor/text2frappe-s2-sbert")
def format_field(field):
    return f"{field}: field in the document"

# Your test cases, e.g. from your big question list
test_cases = [
    ("Sales Invoice", "How many sales invoices are there?", ["name"]),
    ("Sales Invoice", "What is the total amount invoiced?", ["name", "grand_total"]),
    ("Sales Invoice", "How many invoices are paid?", ["name", "status"]),
    ("Sales Invoice", "How many invoices are unpaid?", ["name", "outstanding_amount"]),
    ("Sales Invoice", "What is the latest invoice date?", ["name", "posting_date"]),
    ("Sales Invoice", "Who is the customer on the last invoice?", ["name", "customer_name"]),
    ("Sales Invoice", "What is the total tax amount?", ["name", "total_taxes_and_charges"]),
    ("Sales Invoice", "How many invoices were created today?", ["name", "posting_date"]),
    ("Sales Invoice", "What is the average invoice amount?", ["name", "grand_total"]),
    ("Sales Invoice", "How many invoices have discounts?", ["name", "discount_amount"]),
    ("Supplier", "Which suppliers provide electronic components?", ["name", "supplier_name", "supplier_group"]),
    ("Supplier", "Show suppliers with deliveries pending this week.", ["name", "supplier_name"]),
    ("Supplier", "List suppliers with credit limits above ₹1,00,000.", ["name", "supplier_name"]),
    ("Supplier", "How many purchase orders were made to supplier ABC Corp last month?", ["name", "supplier_name"]),
    ("Supplier", "Show supplier contact person and phone number.", ["name", "supplier_primary_contact", "mobile_no"]),
    ("Supplier", "List suppliers with overdue payments.", ["name", "supplier_name"]),
    ("Supplier", "Which suppliers have been blacklisted?", ["name", "supplier_name", "disabled"]),
    ("Supplier", "Show suppliers associated with purchase invoices this quarter.", ["name", "supplier_name"]),
    ("Supplier", "List suppliers based on their delivery performance.", ["name", "supplier_name"]),
    ("Supplier", "Which suppliers have contracts expiring soon?", ["name", "supplier_name"]),
    ("Supplier", "List all suppliers.", ["name", "supplier_name"]),
    ("Supplier", "Show details of supplier SUP-001.", ["name", "supplier_name"]),
    ("Supplier", "What is the contact information for supplier ABC Corp?", ["name", "supplier_name", "supplier_primary_contact", "mobile_no", "email_id"]),
    ("Supplier", "List suppliers from India.", ["name", "supplier_name", "country"]),
    ("Supplier", "Show supplier addresses.", ["name", "supplier_name", "supplier_primary_address"]),
    ("Supplier", "How many suppliers are currently active?", ["name", "supplier_name", "disabled"]),
    ("Supplier", "List suppliers with pending purchase orders.", ["name", "supplier_name"]),
    ("Supplier", "Show suppliers with the highest rating.", ["name", "supplier_name"]),
    ("Supplier", "What are the payment terms for supplier XYZ Ltd?", ["name", "supplier_name", "payment_terms"]),
    ("Supplier", "List suppliers by industry type.", ["name", "supplier_name", "supplier_group"]),
    ("Employee", "Who are the employees in the sales department?", ["name", "employee_name", "department"]),
    ("Employee", "What is the annual CTC for employee EMP-0012?", ["name", "employee_name", "ctc"]),
    ("Employee", "Show the monthly salary for all employees.", ["name", "employee_name", "ctc"]),
    ("Employee", "List employees with a salary greater than ₹50,000.", ["name", "employee_name", "ctc"]),
    ("Employee", "What is the total salary expense for this month?", ["name", "ctc"]),
    ("Employee", "Which employees have received a salary increase this year?", ["name", "employee_name", "ctc"]),
    ("Employee", "Show the salary mode (e.g., bank transfer, cheque) for employees.", ["name", "employee_name", "salary_mode"]),
    ("Employee", "List employees whose CTC is below ₹30,000.", ["name", "employee_name", "ctc"]),
    ("Employee", "How many leave days has employee EMP-0005 taken this year?", ["name", "employee_name"]),
    ("Employee", "Show leave balance for all employees.", ["name", "employee_name"]),
    ("Employee", "Which employees are currently on leave?", ["name", "employee_name"]),
    ("Employee", "List employees with pending leave approvals.", ["name", "employee_name"]),
    ("Employee", "How many sick leave days were taken last month?", ["name"]),
    ("Employee", "Show leave type details for employee EMP-0010.", ["name", "employee_name"]),
    ("Employee", "Which employees have exhausted their annual leave quota?", ["name", "employee_name"]),
    ("Item", "List all items in stock.", ["name", "item_name", "item_code"]),
    ("Item", "Show details of item ITEM-0001.", ["name", "item_code", "item_name"]),
    ("Item", "What is the price of item ITEM-0050?", ["name", "item_code", "standard_rate"]),
    ("Item", "How many units of ITEM-003 are available?", ["name", "item_code"]),
    ("Item", "List items with low stock levels.", ["name", "item_code", "item_name"]),
    ("Item", "Show items belonging to the Electronics category.", ["name", "item_code", "item_name", "item_group"]),
    ("Item", "What is the warranty period for item ITEM-002?", ["name", "item_code", "warranty_period"]),
    ("Item", "List all stock items.", ["name", "item_code", "item_name", "is_stock_item"]),
    ("Item", "Show supplier details for item ITEM-007.", ["name", "item_code", "supplier_items"]),
    ("Item", "Which items have a reorder level below 10?", ["name", "item_code", "safety_stock"]),
    ("Sales Invoice", "What is the total invoiced amount for last month?", ["name", "grand_total", "posting_date"]),
    ("Employee", "What is the sum of all employee salaries?", ["name", "ctc"]),
    ("Sales Invoice", "Show all sales invoices with their status.", ["name", "status"]),
    ("Sales Invoice", "What is the status of sales invoice SI-0001?", ["name", "status"]),
    ("Sales Invoice", "What is the total invoiced amount for last month?", ["name", "grand_total", "posting_date"]),
    ("Sales Invoice", "Show all invoices created by a specific sales team member.", ["name", "sales_team"]),
    ("Sales Invoice", "Which invoices have discounts applied?", ["name", "discount_amount"]),
    ("Sales Invoice", "Show invoices with payments pending.", ["name", "outstanding_amount"]),
    ("Employee", "Which employees joined in the last quarter?", ["name", "employee_name", "date_of_joining"]),
    ("Employee", "Show contact details for employee EMP-0002.", ["name", "employee_name", "personal_email", "cell_number", "employee_number"]),
    ("Employee", "List employees who are currently inactive.", ["name", "employee_name", "status"]),
    ("Sales Order", "Show total quantity ordered this month.", ["name", "total_qty", "transaction_date"]),
    ("Sales Order", "Which sales orders have been delivered?", ["name", "delivery_status"]),
    ("Sales Order", "Show all sales orders with discounts applied.", ["name", "discount_amount"]),
    ("Sales Order", "List sales orders pending billing.", ["name", "billing_status"]),
    ("Item", "List all stock items available.", ["name", "item_code", "item_name", "is_stock_item"]),
    ("Item", "Which items are classified under Electronics group?", ["name", "item_code", "item_group"]),
    ("Item", "Show items with low stock levels.", ["name", "item_code", "item_name"]),
    ("Item", "List items supplied by vendor XYZ.", ["name", "item_code", "supplier_items"]),
    ("Item", "What products are available in the Electronics category?", ["name", "item_code", "item_name", "item_group"]),
    ("Item", "Show all stock currently available.", ["name", "item_code", "item_name", "is_stock_item"]),
    ("Item", "List products with low inventory levels.", ["name", "item_code", "item_name"]),
    ("Item", "Which products need to be reordered?", ["name", "item_code", "item_name", "safety_stock"]),
    ("Item", "Show details for SKU 12345.", ["name", "item_code", "item_name"]),
    ("Item", "What is the price of the blue widget?", ["name", "item_name", "standard_rate"]),
    ("Item", "How many units are in stock for each product?", ["name", "item_code", "item_name"]),
    ("Item", "List all products with a warranty period over 1 year.", ["name", "item_code", "warranty_period"]),
    ("Item", "Show the supplier for the red chair.", ["name", "item_name", "supplier_items"]),
    ("Item", "Which products belong to the 'Furniture' group?", ["name", "item_code", "item_name", "item_group"]),
    ("Supplier", "Which suppliers provide electronic components?", ["name", "supplier_name", "supplier_group"]),
    ("Supplier", "Show suppliers with deliveries pending this week.", ["name", "supplier_name"]),
    ("Supplier", "List suppliers with credit limits above ₹1,00,000.", ["name", "supplier_name"]),
    ("Supplier", "How many purchase orders were made to supplier ABC Corp last month?", ["name", "supplier_name"]),
    ("Supplier", "Show supplier contact person and phone number.", ["name", "supplier_name", "supplier_primary_contact", "mobile_no"]),
    ("Supplier", "List suppliers with overdue payments.", ["name", "supplier_name"]),
    ("Supplier", "Which suppliers have been blacklisted?", ["name", "supplier_name", "disabled"]),
    ("Supplier", "Show suppliers associated with purchase invoices this quarter.", ["name", "supplier_name"]),
    ("Supplier", "List suppliers based on their delivery performance.", ["name", "supplier_name"]),
    ("Supplier", "Which suppliers have contracts expiring soon?", ["name", "supplier_name"])


]


for doctype, question, expected_fields in test_cases:
    full_question = f"Doctype: {doctype}\nQuestion: {question}"
    if doctype not in doctype_meta:
        print(f"Skipping test, missing meta fields for doctype: {doctype}")
        continue

    candidate_fields = [format_field(f) for f in doctype_meta[doctype]]

    question_emb = model.encode(full_question, convert_to_tensor=True)
    fields_emb = model.encode(candidate_fields, convert_to_tensor=True)

    scores = cos_sim(question_emb, fields_emb)[0]

    ranked_fields = sorted(zip(doctype_meta[doctype], scores), key=lambda x: x[1], reverse=True)

    print(f"\nQuestion: {question}")
    print("Top ranked fields:")
    for field, score in ranked_fields[:5]:
        print(f"  {field}: {score:.4f}")

    top_fields = [f for f, _ in ranked_fields[:5]]
    hits = [f for f in expected_fields if f in top_fields]

    print(f"Expected fields in top 5: {hits} / {expected_fields}")

In [ ]:
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.util import cos_sim
doctype_meta = {dt: fields_info["fields"] for dt, fields_info in meta.items()}
model = SentenceTransformer("/content/drive/MyDrive/Changai/S2/S2 Model/sbert_13_08_25_5")
def format_field(field):
    return f"{field}: field in the document"

# Your test cases, e.g. from your big question list
test_cases = [
    ("Sales Partner", "Who is the sales partner named 'BrightEdge Corp'?", []),
    ("Sales Partner", "Give name, website link of partner who referred 'NovaTech'?", ["partner_name", "website", "referral_code"]),
    ("Sales Partner", "List all sales partners along with their names and commission percentages.", ["partner_name", "commission_rate"]),
    ("Sales Partner", "Identify the distributor with the partner name 'QatarSuppliers Ltd.'", ["partner_name"]),
    ("Sales Partner", "Show me the names and types of all sales partners listed on the website.", ["partner_name", "partner_type", "show_in_website"]),
    ("Sales Partner", "Which partner with the name 'Omega Resale' has a commission rate above 10%?", ["partner_name", "commission_rate"]),
    ("Sales Partner", "Find all partners where the name contains the word 'Global'.", ["partner_name"]),
    ("Sales Partner", "Give me a list of partner names and referral codes for email marketing.", ["partner_name", "referral_code"]),
    ("Sales Partner", "What is the partner name and their introduction text for 'MavenConnect'?", ["partner_name", "introduction"]),
    ("Sales Partner", "What type of sales partner is 'EliteMark Inc.'—a distributor or a reseller?", ["partner_type", "partner_name"]),
    ("Sales Partner", "Show me all partners who are listed as commission agents.", ["partner_name", "partner_type"]),
    ("Sales Partner", "Which sales partners have the partner type set as affiliate?", ["partner_name", "partner_type"]),
    ("Sales Partner", "Find all dealer-type partners operating in the 'GCC' territory.", ["partner_type", "partner_name", "territory"]),
    ("Sales Partner", "What is the partner type associated with 'ResellXperts Pvt Ltd'?", ["partner_type", "partner_name"]),
    ("Sales Partner", "Do we have any affiliates who referred new customers this quarter?", ["partner_type", "referral_code"]),
    ("Sales Partner", "List all resellers whose websites are publicly visible on our portal.", ["partner_type", "show_in_website", "partner_name"]),
    ("Sales Partner", "Which partner types do we currently work with across all operational territories?", ["territory", "partner_type"]),
    ("Sales Partner", "Give me the names and partner types of every partner earning over 10% commission.", ["partner_name", "commission_rate", "partner_type"]),
    ("Sales Partner", "Between resellers and distributors, which type has the higher count in our system?", ["partner_type"]),
    ("Sales Partner", "Which region is 'Alpha Distributors' assigned to?", ["territory"]),
    ("Sales Partner", "Give me the territory and commission rate for 'Skyline Partners'.", ["territory", "commission_rate"]),
    ("Sales Partner", "In which markets are our affiliate partners most active?", ["territory", "partner_type"]),
    ("Sales Partner", "List partners operating in 'Southeast Asia' and show their partner type.", ["territory", "partner_name", "partner_type"]),
    ("Sales Partner", "Find all sales partners assigned to the 'GCC' territory.", ["territory", "partner_name"]),
    ("Sales Partner", "How many partners are listed under the 'Europe' territory?", ["territory"]),
    ("Sales Partner", "Which regions have the most commission agents listed?", ["territory", "partner_type"]),
    ("Sales Partner", "List the names and territories of all partners visible on our website.", ["partner_name", "territory", "show_in_website"]),
    ("Sales Partner", "What’s the combined list of regions covered by our resellers and distributors?", ["territory", "partner_type"]),
    ("Sales Partner", "Do we have any sales partners working in both Europe and Asia?", ["territory"]),
    ("Sales Partner", "What is the commission rate set for 'ResellXperts Pvt Ltd'?", ["commission_rate"]),
    ("Sales Partner", "List all sales partners with a commission rate above 12%.", ["commission_rate"]),
    ("Sales Partner", "Who are the partners with a commission rate of exactly 10.5%?", ["commission_rate"]),
    ("Sales Partner", "Give me names and commission rates of partners who are resellers.", ["partner_name", "commission_rate", "partner_type"]),
    ("Sales Partner", "Show partners with commission rates above 8% operating in 'South Asia'.", ["commission_rate", "territory", "partner_name"]),
    ("Sales Partner", "Is there a partner with 0% commission rate in the system?", ["commission_rate"]),
    ("Sales Partner", "Find all distributors with commission rates between 5% and 15%.", ["commission_rate", "partner_type"]),
    ("Sales Partner", "Which partner offers the highest commission?", ["commission_rate"]),
    ("Sales Partner", "Do any of the top-performing partners have a commission rate below 5%?", ["commission_rate"]),
    ("Sales Partner", "Show partner names, types, and commission rates for website-visible partners.", ["commission_rate", "partner_type", "partner_name", "show_in_website"]),
    ("Sales Partner", "Can you show the address and contact details linked to 'SummitTrade Inc.'?", ["partner_name", "address_contacts"]),
    ("Sales Partner", "Which sales partners have more than one address or contact entry associated with them?", ["address_contacts"]),
    ("Sales Partner", "List all contact entries found under sales partners based in the UAE.", ["territory", "address_contacts"]),
    ("Sales Partner", "Are there any partners without any contact or address entries listed?", ["address_contacts", "partner_name"]),
    ("Sales Partner", "Fetch the partner name, type, and all contact records for resellers in North America.", ["partner_name", "partner_type", "address_contacts", "territory"]),
    ("Sales Partner", "How many contact records are stored for the partner 'GreenLink Pvt Ltd'?", ["address_contacts", "partner_name"]),
    ("Sales Partner", "Give me a breakdown of sales partners and their total number of linked contact entries.", ["address_contacts", "partner_name"]),
    ("Sales Partner", "Do we have any distributors that haven’t added address contacts yet?", ["address_contacts", "partner_type"]),
    ("Sales Partner", "Show me all partners whose address_contacts field includes both phone and email.", ["address_contacts", "partner_name"]),
    ("Sales Partner", "What contact information exists for partners with active referral codes and visible websites?", ["address_contacts", "referral_code", "show_in_website"]),
    ("Sales Partner", "What is the short address description for 'CoreLink Distributors'?", ["address_desc", "partner_name"]),
    ("Sales Partner", "Show the address_desc field for all listed partners.", ["address_desc"]),
    ("Sales Partner", "Can I get the address description for every distributor we work with?", ["address_desc", "partner_type"]),
    ("Sales Partner", "Which sales partners have their address_desc starting with 'Suite 3A'?", ["address_desc"]),
    ("Sales Partner", "List names and short address descriptions of partners based in Europe.", ["partner_name", "address_desc", "territory"]),
    ("Sales Partner", "Get me address_desc and territory for all commission agents.", ["territory", "address_desc", "partner_name", "partner_type"]),
    ("Sales Partner", "Does 'MarketWide Inc.' have an address description stored?", ["address_desc", "partner_name"]),
    ("Sales Partner", "Which partners don’t have any address_desc provided?", ["address_desc", "partner_name"]),
    ("Sales Partner", "Count the number of partners with 'No. 14' in their address description.", ["address_desc"]),
    ("Sales Partner", "Export the partner name and address_desc of all affiliates in the system.", ["address_desc", "partner_name", "partner_type"]),
    ("Sales Partner", "What is the contact description of 'OptiReach Enterprises'?", ["contact_desc"]),
    ("Sales Partner", "List all sales partners and show their contact descriptions.", ["contact_desc"]),
    ("Sales Partner", "Do we have contact_desc details for every reseller listed?", ["contact_desc"]),
    ("Sales Partner", "Show me names and contact descriptions of distributors in the Middle East.", ["partner_name", "contact_desc"]),
    ("Sales Partner", "Which partner has a contact_desc starting with 'Main Office at'?", ["contact_desc"]),
    ("Sales Partner", "Get the partner name and their contact description for everyone showing on the website.", ["contact_desc", "partner_name"]),
    ("Sales Partner", "Find partners where the contact_desc is missing or incomplete.", ["contact_desc"]),
    ("Sales Partner", "What does the contact description say for 'VibeConnect Limited'?", ["contact_desc"]),
    ("Sales Partner", "How many sales partners do not have contact descriptions?", ["contact_desc"]),
    ("Sales Partner", "Export the partner name, type, and contact description for all affiliates.", ["partner_name", "partner_type", "contact_desc"]),
    ("Sales Partner", "What is the description given for 'Orbit Trade Solutions' in our partner records?", ["description"]),
    ("Sales Partner", "List all partners along with their descriptions.", ["description"]),
    ("Sales Partner", "Which partners include the word 'logistics' in their description?", ["description"]),
    ("Sales Partner", "Can you show the name and description of each partner based in Asia?", ["description", "partner_name"]),
    ("Sales Partner", "Get me the description and partner type for affiliates in our list.", ["description", "partner_type"]),
    ("Sales Partner", "Do we have a partner with 'ecommerce' mentioned in the description?", ["description"]),
    ("Sales Partner", "Compare the descriptions of 'BrightLeaf Group' and 'CoreBridge Alliance'.", ["description"]),
    ("Sales Partner", "Show me names and descriptions of all partners who earn over 15% commission.", ["description", "partner_name"]),
    ("Sales Partner", "What’s the official description of our partner 'LogiMax Pvt Ltd'?", ["description"]),
    ("Sales Partner", "Find all partners who have empty or missing descriptions.", ["description"]),
    ("Sales Partner", "How much commission does 'TradeSphere Global' earn?", ["commission_rate", "partner_name"]),
    ("Sales Partner", "List all sales partners earning more than 10% commission.", ["commission_rate"]),
    ("Sales Partner", "What’s the commission rate for our distributor 'BrightWave Solutions'?", ["commission_rate"]),
    ("Sales Partner", "Show names and commission percentages of all commission agents.", ["partner_name", "partner_type", "commission_rate"]),
    ("Sales Partner", "Find partners earning less than or equal to 5% commission.", ["commission_rate"]),
    ("Sales Partner", "Is there any partner with exactly 12% commission?", ["commission_rate"]),
    ("Sales Partner", "Give me the partner name, website, and commission percentage for all resellers.", ["partner_name", "website", "commission_rate"]),
    ("Sales Partner", "Are distributors earning more commission than affiliates?", ["commission_rate"]),
    ("Sales Partner", "Show me the contact details displayed for 'Orbit Connect Pvt Ltd'.", ["contact_html"]),
    ("Sales Partner", "Which sales partners have formatted contact information available?", ["contact_html"]),
    ("Sales Partner", "Do we store contact HTML for each partner profile?", ["contact_html"]),
    ("Sales Partner", "Get a list of all partners with their HTML-based contact blocks.", ["contact_html"]),
    ("Sales Partner", "Which partners have both their formatted contacts and website links listed?", ["contact_html", "website"]),
    ("Sales Partner", "Give me the contact block and territory for all resellers.", ["contact_html", "territory"]),
    ("Sales Partner", "Find partners who have provided both contact HTML and introduction.", ["contact_html", "introduction"]),
    ("Sales Partner", "Can you give me names and sales targets for all European partners?", ["partner_name", "targets"]),
    ("Sales Partner", "List all distributor-type partners and the targets assigned to them.", ["partner_type", "targets"]),
    ("Sales Partner", "Who are our top-performing partners in terms of meeting targets?", ["partner_name", "targets"]),
    ("Sales Partner", "Compare the targets assigned to resellers vs distributors.", ["partner_type", "targets"]),
    ("Sales Partner", "What is the official website of 'Torotta Avlaye'?", ["website"]),
    ("Sales Partner", "List all partners whose websites end with '.com.qa'.", ["website"]),
    ("Sales Partner", "Do any of our reseller partners have websites listed?", ["website", "partner_type"]),
    ("Sales Partner", "Show partner names and their websites for those visible to the public.", ["partner_name", "website", "show_in_website"]),
    ("Sales Partner", "Which partners have no website provided in their records?", ["website"]),
    ("Sales Partner", "Get me a list of partner websites containing the word 'global'.", ["website"]),
    ("Sales Partner", "Can you find all distributor partners and display their website URLs?", ["partner_type", "website"]),
    ("Sales Partner", "Do we have any affiliate partners with websites ending in '.org'?", ["partner_type", "website"]),
    ("Sales Partner", "Between 'Resell360' and 'OmniPartners', which one has a listed website?", ["partner_name", "website"]),
    ("Sales Partner", "Which partner has the website 'www.synergylink.com'?", ["website", "partner_name"]),
    ("Sales Partner", "What is the referral code assigned to 'Altura Traders'?", ["referral_code"]),
    ("Sales Partner", "List all partners along with their referral_code values.", ["referral_code"]),
    ("Sales Partner", "Which partners don’t have any referral_code provided in their records?", ["referral_code"]),
    ("Sales Partner", "Find partners who have referral codes starting with 'AFF-'.", ["referral_code"]),
    ("Sales Partner", "Does 'Zenith Connect' have a referral_code assigned?", ["referral_code"]),
    ("Sales Partner", "Get the referral_code and website for all affiliate-type partners.", ["referral_code", "website"]),
    ("Sales Partner", "Which sales partners use 'REF-9082-XP' as their referral code?", ["referral_code"]),
    ("Sales Partner", "Export partner_name and referral_code for all partners in Qatar.", ["referral_code", "partner_name"]),
    ("Sales Partner", "Count how many partners have valid referral codes stored.", ["referral_code"]),
    ("Sales Partner", "Which referral_code corresponds to the partner 'BridgeGlobal Inc.'?", ["referral_code"]),
    ("Sales Partner", "What is the route path assigned to 'GlobalLink Solutions' on our website?", ["route"]),
    ("Sales Partner", "List all sales partners whose route starts with '/partners/'.", ["route"]),
    ("Sales Partner", "Show me the partner names and their corresponding web route paths.", ["route"]),
    ("Sales Partner", "Which partners have unique route entries configured for the site?", ["route"]),
    ("Sales Partner", "Do we have any partners using the default route value or an empty one?", ["route"]),
    ("Sales Partner", "Between 'TradeBridge' and 'MarketScope', who has a custom route defined?", ["route"]),
    ("Sales Partner", "Give me all route entries that contain the word 'exclusive'.", ["route"]),
    ("Sales Partner", "Which sales partner has the URL route '/partner-profile/gcc-distributor'?", ["route"]),
    ("Sales Partner", "Export all visible partners on the website along with their route links.", ["route"]),
    ("Sales Partner", "Are there any route fields duplicated across different partners?", ["route"]),
    ("Sales Partner", "What is the external website link provided for 'BlueOcean Vendors'?", ["partner_website"]),
    ("Sales Partner", "List the partner_website URLs of all partners visible to customers.", ["partner_website"]),
    ("Sales Partner", "Can I view the official external sites for all commission agents?", ["partner_website"]),
    ("Sales Partner", "Get partner names and partner_website links for reseller-type entries.", ["partner_website", "partner_name"]),
    ("Sales Partner", "Which partners do not have a partner_website listed yet?", ["partner_website"]),
    ("Sales Partner", "Are there any affiliate partners whose websites contain 'tradehub'?", ["partner_website"]),
    ("Sales Partner", "Between 'NetBridge Ltd' and 'MarketSpark', who has a valid partner_website?", ["partner_website"]),
    ("Sales Partner", "Show all partner_website URLs that end in '.tech' domain.", ["partner_website"]),
    ("Sales Partner", "Do any of the partners have a link to 'https://affiliateworld.org' in their records?", ["partner_website"]),
    ("Sales Partner", "Fetch the partner_website and contact description for partners in South America.", ["partner_website", "contact_desc"]),
    ("Sales Partner", "What is the introduction note for 'Orbit Traders' in our sales partner list?", ["introduction"]),
    ("Sales Partner", "Show me all partners along with their introduction texts.", ["introduction"]),
    ("Sales Partner", "Which sales partners have introduction messages starting with 'Leading exporter'?", ["introduction"]),
    ("Sales Partner", "List the name and introduction for every commission agent.", ["introduction", "partner_name"]),
    ("Sales Partner", "Which partners don't have any introduction filled in?", ["introduction"]),
    ("Sales Partner", "Provide all introductions for partners in the European region.", ["introduction"]),
    ("Sales Partner", "Export a list of reseller-type partners and their introduction fields.", ["introduction", "partner_type"]),
    ("Sales Partner", "Check if 'GlobeX Trade Corp' has an introduction field entered.", ["introduction"]),
    ("Sales Partner", "How many sales partners have included an introduction message?", ["introduction"]),
    ("Supplier", "What is the naming pattern used to generate supplier IDs?", ["naming_series"]),
    ("Supplier", "Can you show me how supplier codes are automatically generated?", ["naming_series"]),
    ("Supplier", "Which document controls the prefix used for new supplier entries?", ["naming_series"]),
    ("Supplier", "Find out how supplier numbers are structured in the system.", ["naming_series"]),
    ("Supplier", "Where is the supplier ID format like 'SUP-.YYYY.-' defined?", ["naming_series"]),
    ("Supplier", "How are suppliers named when records are created?", ["naming_series"]),
    ("Supplier", "What format is used for supplier auto-generated names?", ["naming_series"]),
    ("Supplier", "Is there a setting that defines how new supplier records are named?", ["naming_series"]),
    ("Supplier", "Which doctype includes the naming series for vendors?", ["naming_series"]),
    ("Supplier", "What is the naming pattern for the vendors?", ["naming_series"]),
    ("Supplier", "Who is the vendor with code SUP-00015?", ["supplier_name"]),
    ("Supplier", "Pull up a list of suppliers based in Germany for vendor negotiations.", ["supplier_name"]),
    ("Supplier", "Which supplier is linked to purchase order PO-332?", ["supplier_name"]),
    ("Supplier", "Show me the supplier name for internal transport services.", ["supplier_name"]),
    ("Supplier", "Get the supplier name for 'Metallic Edge Corp'.", ["supplier_name"]),
    ("Supplier", "Get the mobile number for 'Metallic Edge Corp'.", ["mobile_no"]),
    ("Supplier", "Find the supplier that uses the email 'supplies@techlink.io'.", ["supplier_name"]),
    ("Supplier", "Get me the supplier name for vendors that are marked frozen in the system.", ["supplier_name"]),
    ("Supplier", "Get me the default currency for vendors that are marked frozen in the system.", ["default_currency"]),
    ("Supplier", "Check the supplier name for vendor ID SUP-00021.", ["supplier_name"]),
    ("Supplier", "Check the primary contact for vendor ID SUP-00021.", ["supplier_primary_contact"]),
    ("Supplier", "List all active suppliers under the 'Raw Material' group.", ["supplier_name"]),
    ("Supplier", "I need supplier names for transport service providers.", ["supplier_name"]),
    ("Supplier", "I need email IDs for transport service providers.", ["email_id"]),
    ("Supplier", "Which doctype includes the naming_series for vendors?", ["naming_series"]),
    ("Supplier", "Which country is 'Atlantic Trade Co.' registered in?", ["country"]),
    ("Supplier", "List all suppliers located in the United Arab Emirates.", ["country"]),
    ("Supplier", "Get suppliers from India and Bangladesh for textile sourcing.", ["country"]),
    ("Supplier", "Show me vendors registered in countries outside the EU.", ["country"]),
    ("Supplier", "Do we have any suppliers from South Korea?", ["country"]),
    ("Supplier", "Filter all supplier entries that mention 'Germany' as their country.", ["country"]),
    ("Supplier", "How many suppliers are currently registered in China?", ["country"]),
    ("Supplier", "Export list of suppliers along with their country of operation.", ["country", "supplier_name"]),
    ("Supplier", "Which vendors are based in Australia and New Zealand?", ["country"]),
    ("Supplier", "Who are our suppliers from Latin American countries?", ["country"]),
    ("Supplier", "Which supplier group does 'Fresh Agro Supplies' fall under?", ["supplier_group"]),
    ("Supplier", "List all suppliers categorized under 'Raw Material Vendors'.", ["supplier_group"]),
    ("Supplier", "How many suppliers do we have in the 'Importers' group?", ["supplier_group"]),
    ("Supplier", "Get the supplier names and their groups for all active vendors from India.", ["supplier_group", "supplier_name"]),
    ("Supplier", "Show the group classification for suppliers that are transport service providers.", ["supplier_group"]),
    ("Supplier", "Do we have any local suppliers under the 'Capital Equipment' group?", ["supplier_group"]),
    ("Supplier", "Pull all supplier records under the 'Utilities & Services' group with payment terms set.", ["supplier_group", "payment_terms"]),
    ("Supplier", "For each supplier group, show the default currency and the number of associated suppliers.", ["supplier_group", "default_currency"]),
    ("Supplier", "Are there any suppliers who also handle transportation services for us?", ["is_transporter"]),
    ("Supplier", "Which vendors are marked as transport providers for our delivery operations?", ["is_transporter"]),
    ("Supplier", "Can you pull up a list of active suppliers who also offer transportation?", ["is_transporter"]),
    ("Supplier", "I need the names of companies that supply goods and manage their own transport.", ["is_transporter"]),
    ("Supplier", "Which suppliers in our system provide transport services along with their products?", ["is_transporter"]),
    ("Supplier", "Can you find suppliers who handle invoicing but don’t offer transportation?", ["is_transporter"]),
    ("Supplier", "Which vendors don’t provide transport services but outsource deliveries?", ["is_transporter"]),
    ("Supplier", "Show me all transport providers from our supplier database that work with construction projects.", ["is_transporter"]),
    ("Supplier", "Pull up supplier names along with their transport service status for audit purposes.", ["is_transporter"]),
    ("Supplier", "Which of our suppliers also operate as logistics partners?", ["is_transporter"]),
    ("Supplier", "Which vendors have uploaded a profile image in the system?", ["image"]),
    ("Supplier", "Show me the supplier name and image for all internal suppliers.", ["supplier_name", "image"]),
    ("Supplier", "List suppliers along with their logos for catalog publishing.", ["supplier_name", "image"]),
    ("Supplier", "Do we have image records for vendors categorized as 'Transport Services'?", ["image"]),
    ("Supplier", "Get the name, type, and image of all disabled suppliers.", ["supplier_name", "supplier_type", "image"]),
    ("Supplier", "Find suppliers who haven't uploaded their business image yet.", ["image"]),
    ("Supplier", "Pull the image and default currency info for active suppliers.", ["image", "default_currency"]),
    ("Supplier", "Which vendors have profile pictures and valid email addresses?", ["image", "email_id"]),
    ("Supplier", "Do any suppliers have images associated with their company profiles?", ["image"]),
    ("Supplier", "Get me the profile pictures and contact details of key international vendors.", ["image", "mobile_no", "email_id"]),
    ("Supplier", "What is the default bank account associated with each supplier?", ["default_bank_account"]),
    ("Supplier", "Get supplier names along with their default bank accounts.", ["supplier_name", "default_bank_account"]),
    ("Supplier", "Which internal suppliers have a default bank account listed?", ["is_internal_supplier", "default_bank_account"]),
    ("Supplier", "Show me the name, currency, and default bank account for all active suppliers.", ["supplier_name", "default_currency", "default_bank_account", "disabled"]),
    ("Supplier", "Find suppliers who have no default bank account setup but are not frozen.", ["default_bank_account", "is_frozen"]),
    ("Supplier", "Pull suppliers with both a valid default bank account and price list.", ["default_bank_account", "default_price_list"]),
    ("Supplier", "List suppliers that are not disabled and have bank details set for default transactions.", ["default_bank_account", "disabled"]),
    ("Supplier", "Show me the suppliers who represent companies and have default bank accounts set.", ["represents_company", "default_bank_account"]),
    ("Supplier", "Export supplier names and their bank accounts used for automatic payments.", ["supplier_name", "default_bank_account"]),
    ("Supplier", "Do any vendors lack a default bank account for incoming payment setups?", ["default_bank_account"]),
    ("Sales Invoice", "What is the average invoice amount this month?", ["grand_total", "posting_date", "docstatus", "name"]),
    ("Sales Invoice", "How many sales invoices marked as returns were recorded this quarter?", ["is_return", "posting_date", "docstatus", "name"]),
    ("Sales Invoice", "What is the total revenue from sales invoices this year?", ["grand_total", "posting_date", "docstatus", "name"]),
    ("Sales Invoice", "Which customer has the highest total invoice amount this year?", ["customer", "grand_total", "posting_date", "docstatus"]),
    ("Sales Invoice", "How many invoices were submitted last week?", ["posting_date", "docstatus", "name"]),
    ("Sales Invoice", "List all submitted Sales Invoices from this month.", ["posting_date", "docstatus", "name"]),
    ("Sales Invoice", "Show total invoice amount for each customer this year.", ["customer", "grand_total", "posting_date"]),
    ("Sales Invoice", "How many Sales Invoices created last quarter.", ["docstatus"]),
    ("Sales Invoice", "List all invoices with status 'Overdue'.", ["due_date", "outstanding_amount", "docstatus", "name"]),
    ("Sales Invoice", "What is the total revenue from submitted invoices this fiscal year?", ["grand_total", "posting_date", "docstatus", "name"]),
    ("Sales Invoice", "Show average invoice amount per month for the current year.", ["grand_total", "posting_date", "docstatus", "name"]),
    ("Sales Invoice", "Which invoices were marked as returns this month?", ["customer", "is_return", "posting_date", "docstatus"]),
    ("Sales Invoice", "How many invoices were cancelled last week?", ["name", "docstatus", "posting_date"]),
    ("Sales Invoice", "List all invoices with total amount greater than ₹50,000.", ["name", "grand_total"]),
    ("Sales Invoice", "Show all invoices linked to customer 'Acme Corp'.", ["name", "customer"]),
    ("Sales Invoice", "List invoices with due date in the next 7 days.", ["due_date"]),
    ("Sales Invoice", "What is the total invoice amount for this month?", ["grand_total"]),
    ("Sales Invoice", "Which company issued the invoice?", ["company"]),
    ("Sales Invoice", "Show all draft invoices created this week.", ["name", "docstatus", "creation"]),
    ("Sales Invoice", "Which invoices were paid this month?", ["name", "paid_amount", "posting_date"]),
    ("Sales Invoice", "How many Sales Invoices were created this month?", ["name", "creation"]),
    ("Sales Invoice", "How many invoices are currently in draft status?", ["docstatus"]),
    ("Sales Invoice", "How many invoices were cancelled this quarter?", ["docstatus", "posting_date"]),
    ("Sales Invoice", "How many invoices were marked as returns this year?", ["is_return", "posting_date"]),
    ("Sales Invoice", "How many invoices were created today?", ["posting_date", "docstatus"]),
    ("Sales Invoice", "How many invoices are overdue as of now?", ["due_date", "outstanding_amount", "docstatus"]),
    ("Sales Invoice", "How many invoices were created for customer 'Acme Corp' this year?", ["customer", "posting_date"]),
    ("Sales Invoice", "How many invoices have a total amount above ₹1,00,000?", ["rounded_total"]),
    ("Customer", "How many customers belong to the 'Retail' customer group?", ["customer_group"]),
    ("Customer", "Show all customers in the 'West Region' territory.", ["territory"]),
    ("Customer", "List all customers created in the last 30 days.", ["creation"]),
    ("Customer", "What is the email address of customer 'Acme Corp'?", ["email_id"]),
    ("Customer", "Which customers have a credit limit above ₹1,00,000?", ["credit_limits"]),
    ("Customer", "Show customers who are marked as internal customers.", ["is_internal_customer"]),
    ("Customer", "Show the primary contact details for customer 'Acme Corp'.", ["customer_primary_contact"]),
    ("Customer", "How many customers are currently disabled?", ["disabled"]),
    ("Customer", "List all customers managed by sales team 'Team A'.", ["sales_team"]),
    ("Customer", "How many customers have a loyalty program assigned?", ["loyalty_program"]),
    ("Customer", "Show all customers.", ["name"]),
    ("Item", "Show me details of the item with ID 'ITEM-0001'.", ["name", "item_code", "stock_uom", "item_group"]),
    ("Item", "How many stock items are currently below their reorder levels?", ["reorder_levels"]),
    ("Item", "How many service items do we have in our catalog?", ["is_stock_item"]),
    ("Item", "Which items have a warranty period longer than 12 months?", ["name", "warranty_period"]),
    ("Item", "List items supplied by 'ABC Supplies' with a last purchase rate above 500.", ["supplier_items", "last_purchase_rate"]),
    ("Item", "Which items have variants available and who is the default manufacturer?", ["has_variants", "default_item_manufacturer"]),
    ("Item", "Show items with batch numbers and expiry dates that have safety stock set.", ["has_batch_no", "has_expiry_date", "safety_stock"]),
    ("Item", "Which items require inspection before purchase and have serial numbers?", ["inspection_required_before_purchase", "has_serial_no"]),
    ("Item", "List items with maximum discount limits and their standard selling price.", ["max_discount", "standard_rate"]),
    ("Item", "How many items allow negatives stock to be recorded?", ["allow_negative_stock"]),
    ("Item", "How many items have their stock unit of measure as 'Pieces'?", ["stock_uom"]),
    ("Item", "How many items have a default sales unit set?", ["sales_uom"]),
    ("Item", "How many items are currently disabled in the system?", ["disabled"]),
    ("Item", "How many items are currently active and available for sale?", ["docstatus"]),
    ("Item", "How many items have an assigned brand name?", ["brand"]),
    ("Sales Order", "Show details of the sales order with ID 'SO-0001'.", ["name", "customer", "delivery_date"]),
    ("Sales Order", "How many sales orders were created this month?", ["docstatus"]),
    ("Sales Order", "How many sales orders have been fully delivered?", ["delivery_status"]),
    ("Sales Order", "How many sales orders have been billed but not yet delivered?", ["billing_status"]),
    ("Sales Order", "What is the total amount for sales order 'SO-0001'?", ["grand_total", "name"]),
    ("Sales Order", "List sales orders for customer 'Acme Corp' due for delivery next week.", ["customer", "delivery_date"]),
    ("Sales Order", "Show sales orders created by the sales partner 'John Doe'.", ["sales_partner", "name"]),
    ("Sales Order", "How many sales orders have discounts applied this quarter?", ["discount_amount", "docstatus"]),
    ("Sales Order", "Which sales orders have been amended from previous orders?", ["amended_from"]),
    ("Sales Order", "Can you pull up the order ID for the sales order that Acme Corp placed last week?", ["name"]),
    ("Sales Order", "Who’s the customer on sales order SO-0005?", ["customer"]),
    ("Sales Order", "When is the delivery scheduled for order SO-0020?", ["delivery_date"]),
    ("Sales Order", "How many sales orders got confirmed this quarter?", ["docstatus"]),
    ("Sales Order", "Can you tell me how many orders have been marked as fully delivered?", ["delivery_status"]),
    ("Sales Order", "How many orders are fully billed but still waiting on delivery?", ["billing_status"]),
    ("Sales Order", "What’s the total amount for sales order SO-0100?", ["grand_total"]),
    ("Sales Order", "What’s the unique ID for that sales order with the big total?", ["name"]),
    ("Sales Order", "Show me all sales orders from Beta Ltd that are due for delivery next month.", ["customer"]),
    ("Sales Order", "What’s the expected delivery date for all orders from Beta Ltd due next month?", ["delivery_date"]),
    ("Sales Order", "Which sales orders did Alice Johnson handle last quarter?", ["sales_partner"]),
    ("Sales Order", "What are the order IDs for the sales orders handled by Alice Johnson last quarter?", ["name"]),
    ("Sales Order", "How many orders had discounts applied last month?", ["discount_amount"]),
    ("Sales Order", "Can you list sales orders that were changed or updated from earlier ones?", ["amended_from"]),
    ("Sales Order", "Can you find the unique order ID for the sales order placed by customer 'Global Tech' last Monday?", ["name"]),
    ("Sales Order", "Who is the customer listed on sales order SO-0456?", ["customer"]),
    ("Sales Order", "What’s the delivery date for order SO-0456?", ["delivery_date"]),
    ("Sales Order", "How many orders did we confirm in the last three months?", ["docstatus"]),
    ("Sales Order", "How many sales orders have been fully delivered so far?", ["delivery_status"]),
    ("Sales Order", "Which orders are fully billed but still waiting on delivery?", ["billing_status"]),
    ("Sales Order", "What was the total value for sales order SO-0789?", ["grand_total"]),
    ("Sales Order", "Can you provide the order ID for that high-value sales order last week?", ["name"]),
    ("Sales Order", "Show me all sales orders from 'TechWorld' expected to be delivered next month.", ["customer"]),
    ("Sales Order", "What delivery dates are scheduled for orders from 'TechWorld' next month?", ["delivery_date"]),
    ("Sales Order", "Which sales orders did 'John Smith' handle last quarter?", ["sales_partner"]),
    ("Sales Order", "Can you give me the order IDs for sales handled by 'John Smith' last quarter?", ["name"]),
    ("Sales Order", "How many orders included a discount last month?", ["discount_amount"]),
    ("Sales Order", "How many orders were finalized last month regardless of discount?", ["docstatus"]),
    ("Purchase Invoice", "Can you find the purchase invoice ID for the invoice raised by supplier 'Global Supplies' last week?", ["supplier", "posting_date"]),
    ("Purchase Invoice", "What is the due date for payment on purchase invoice PI-0023?", ["due_date", "name"]),
    ("Purchase Invoice", "Who supplied the goods in purchase invoice PI-0023?", ["name", "supplier"]),
    ("Purchase Invoice", "How many purchase invoices were posted last month?", ["posting_date"]),
    ("Purchase Invoice", "How many purchase invoices are currently unpaid?", ["is_paid"]),
    ("Purchase Invoice", "What is the monthly trend of purchase expenses this year?", ["name", "posting_date"]),
    ("Purchase Invoice", "What is the total amount for purchase invoice PI-0050?", ["name", "grand_total"]),
    ("Purchase Invoice", "How many purchase invoices were returned this quarter?", ["is_return"]),
    ("Purchase Invoice", "Can you list the purchase invoice IDs that were amended from previous invoices?", ["amended_from"]),
    ("Purchase Invoice", "Which purchase invoices have outstanding amounts as of today?", ["outstanding_amount"]),
    ("Purchase Invoice", "Show me purchase invoices from 'Grant Plastics Ltd' with payment due within 30 days.", ["name", "supplier", "due_date", "docstatus"]),
    ("Purchase Invoice", "What is the total amount spent on purchases this month?", ["name", "grand_total", "posting_date", "docstatus"]),
    ("Sales Order", "List all sales orders that were updated or amended from earlier orders.", ["name", "amended_from"]),
    ("Employee", "What is the salary of John Doe?", ["name", "ctc"]),
    ("Employee", "Show me the CTC of all employees.", ["ctc", "name"]),
    ("Employee", "Which employee has the highest salary?", ["ctc", "name"]),
    ("Employee", "How many leaves has John Doe taken this year?", ["leave_encashed", "name"]),
    ("Sales Invoice", "How many sales invoices were created last month?", ["posting_date", "name"]),
    ("Sales Invoice", "What is the total outstanding amount for all sales invoices?", ["outstanding_amount", "name"]),
    ("Sales Invoice", "List all sales invoices with their status and invoice name.", ["status", "name"]),
    ("Purchase Invoice", "Can you show me all purchase invoices from supplier 'ABC Traders' for March?", ["name", "supplier", "posting_date"]),
    ("Purchase Invoice", "What is the outstanding amount for purchase invoice PINV-2024-00010?", ["name", "outstanding_amount"]),
    ("Purchase Invoice", "List purchase invoices with status 'Paid' along with their total amounts.", ["name", "status", "grand_total"]),
    ("Purchase Invoice", "How many purchase invoices were submitted last quarter?", ["name", "posting_date", "docstatus"]),
    ("Purchase Invoice", "What's the total value of purchase invoices raised in this financial year?", ["name", "grand_total", "posting_date"]),
    ("Purchase Invoice", "Which supplier billed us the most this year?", ["name", "supplier", "grand_total"]),
    ("Purchase Invoice", "Show purchase invoices where payment is overdue by more than 30 days.", ["name", "outstanding_amount", "posting_date"]),
    ("Purchase Invoice", "Show all draft purchase invoices that haven’t been submitted.", ["name", "docstatus"]),
    ("Purchase Invoice", "What tax amount was charged on purchase invoice PINV-2024-00345?", ["name", "taxes_and_charges"]),
    ("Purchase Invoice", "List purchase invoices created this month with totals above ₹50,000.", ["name", "grand_total", "posting_date"]),
    ("Sales Invoice", "What is the total revenue this month?", ["name", "posting_date", "grand_total", "docstatus"]),
    ("Sales Invoice", "Which customer generated the highest revenue this quarter?", ["name", "customer", "posting_date", "grand_total", "docstatus"]),
    ("Sales Invoice", "Show average invoice value by customer.", ["name", "customer", "grand_total", "docstatus"]),
    ("Sales Invoice", "List all invoices above ₹1,00,000.", ["name", "grand_total", "posting_date", "customer", "docstatus"]),
    ("Sales Invoice", "What is the monthly sales trend this year?", ["name", "posting_date", "grand_total", "docstatus"]),
    ("Sales Invoice", "Show revenue breakdown by territory.", ["name", "territory", "posting_date", "grand_total", "docstatus"]),
    ("Sales Invoice", "How many invoices were created this week?", ["name", "posting_date", "docstatus"]),
    ("Sales Invoice", "Which invoices are still unpaid?", ["name", "outstanding_amount", "docstatus"]),
    ("Sales Invoice", "List overdue invoices by customer.", ["name", "due_date", "customer", "docstatus"]),
    ("Sales Invoice", "What is the total outstanding amount?", ["name", "outstanding_amount", "docstatus"]),
    ("Sales Invoice", "Show aging report for receivables.", ["name", "due_date", "outstanding_amount", "docstatus"]),
    ("Purchase Invoice", "What is the total purchase expense this month?", ["name", "posting_date", "grand_total", "docstatus"]),
    ("Purchase Invoice", "Which supplier invoiced us the most this quarter?", ["name", "supplier", "posting_date", "grand_total", "docstatus"]),
    ("Purchase Invoice", "Show average invoice value by supplier.", ["name", "supplier", "grand_total", "docstatus"]),
    ("Purchase Invoice", "List all purchase invoices above ₹50,000.", ["name", "grand_total", "posting_date", "supplier", "docstatus"]),
    ("Purchase Invoice", "Which supplier has the most unpaid invoices?", ["name", "supplier", "outstanding_amount", "docstatus"]),
    ("Purchase Invoice", "How many purchase invoices were created this week?", ["name", "posting_date", "docstatus"]),
    ("Employee", "Which department has the highest average salary?", ["name", "department", "ctc"]),
    ("Employee", "How many employees joined in the last 6 months?", ["name", "date_of_joining"]),
    ("Employee", "What is the total salary cost for each branch?", ["name", "branch", "ctc"]),
    ("Employee", "Which designation has the highest average CTC?", ["name", "designation", "ctc"]),
    ("Employee", "Show average salary by gender.", ["name", "gender", "ctc"]),
    ("Employee", "List top 10 highest-paid employees.", ["name", "ctc"]),
    ("Employee", "Which department has the widest salary range?", ["name", "department", "ctc"]),
    ("Employee", "What is the median salary across all employees?", ["name", "ctc"]),
    ("Employee", "Compare average salary between confirmed and probation employees.", ["name", "status", "ctc"]),
    ("Employee", "List all employees with CTC above ₹1,00,000.", ["name", "ctc"]),
    ("Employee", "Show salary distribution by marital status.", ["name", "marital_status", "ctc"]),
    ("Purchase Invoice", "How many purchase invoices were created last month?", ["posting_date"]),
    ("Purchase Invoice", "What is the total amount of purchase invoices this quarter?", ["name", "total"]),
    ("Purchase Invoice", "What is the exchange gain/loss on purchase invoice PINV-0005?", ["name", "unrealized_profit_loss_account"]),
    ("Purchase Invoice", "List all purchase invoices with their supplier names.", ["name", "supplier_name"]),
    ("Purchase Invoice", "What is the outstanding amount for purchase invoice PINV-0010?", ["name", "outstanding_amount"]),
    ("Purchase Invoice", "What is the total tax amount for purchase invoices this quarter?", ["name", "total_taxes_and_charges"]),
    ("Purchase Invoice", "How many purchase invoices were canceled this year?", ["name", "posting_date"]),
    ("Purchase Invoice", "What is the earliest posting date of a purchase invoice this year?", ["name", "posting_date"]),
    ("Purchase Invoice", "List all purchase invoices with outstanding amounts greater than ₹20,000.", ["name", "outstanding_amount"]),
    ("Purchase Invoice", "What is the payment status of purchase invoice PINV-0003?", ["name", "status"]),
    ("Purchase Invoice", "How much was spent on raw materials last quarter?", ["name", "raw_materials_supplied"]),
    ("Purchase Invoice", "Which supplier had the highest invoice total last year?", ["name", "supplier"]),
    ("Purchase Invoice", "What is the average purchase cost per item category this year?", ["name", "items"]),
    ("Purchase Invoice", "How much exchange gain/loss occurred each quarter for the last 2 years?", ["name", "unrealized_profit_loss_account"]),
    ("Employee", "How many employees joined last month?", ["date_of_joining"]),
    ("Employee", "What is the total salary cost for all employees?", ["ctc"]),
    ("Employee", "What is the status of employee EMP-2025-00005?", ["name", "status"]),
    ("Employee", "How many employees left the company this quarter?", ["relieving_date"]),
    ("Employee", "What is the total salary paid to employees this year?", ["ctc"]),
    ("Employee", "How many employees were confirmed last week?", ["final_confirmation_date"]),
    ("Sales Order", "How many sales orders were created last month?", ["creation"]),
    ("Sales Order", "What is the total outstanding amount for all sales orders?", ["outstanding_amount", "name"]),
    ("Sales Order", "What is the status of sales order SO-2025-00005?", ["status", "name"]),
    ("Sales Order", "How many sales orders were canceled this quarter?", ["status"]),
    ("Project", "How many projects are currently active?", ["is_active"]),
    ("Project", "What is the total budget spent per project?", ["name", "total_costing_amount"]),
    ("Project", "Which projects are behind schedule?", ["name", "expected_end_date", "actual_end_date", "status"]),
    ("Project", "What is the resource allocation across projects?", ["name", "users"]),
    ("Task", "How many tasks were completed last month?", ["completed_on"]),
    ("Task", "What is the total billing amount per task?", ["name", "total_billing_amount"]),
    ("Employee", "How many employees have not completed mandatory training?", ["training_name", "employee"]),
    ("Payment Entry", "What is the total outstanding receivables as of today?", ["get_outstanding_invoices"]),
    ("Journal Entry", "What is the total outstanding receivables as of today?", ["get_outstanding_invoices"]),
    ("Journal Entry", "How many invoices are overdue for payment?", ["due_date"]),
    ("Payment Entry", "How many invoices are overdue for payment?", ["get_outstanding_invoices"]),
    ("Journal Entry", "What is the cash flow forecast for the next quarter?", ["total_debit"]),
    ("Payment Entry", "What is the cash flow forecast for the next quarter?", ["paid_amount"]),
    ("Journal Entry", "List all journal entries for a given period.", ["posting_date", "name"]),
    ("Account", "What is the profit and loss statement for the last financial year?", ["root_type"]),
    ("Journal Entry", "What is the profit and loss statement for the last financial year?", ["total_debit"]),
    ("Journal Entry", "How much tax was collected/paid last quarter?", ["tax_withholding_category"]),
    ("Payment Entry", "How much tax was collected/paid last quarter?", ["total_taxes_and_charges"]),
    ("Account", "Which accounts have the highest debit/credit balance?", ["balance_must_be"]),
    ("Journal Entry", "Which accounts have the highest debit/credit balance?", ["total_debit"]),
    ("Item", "What is the current stock level of key items?", ["name", "item_code"]),
    ("Stock Ledger Entry", "What is the current stock level of key items?", ["name", "qty_after_transaction"]),
    ("Item", "Which items are nearing reorder level?", ["name", "reorder_levels"]),
    ("Warehouse", "What is the total stock value by warehouse?", ["name", "warehouse_name"]),
    ("Stock Ledger Entry", "What is the total stock value by warehouse?", ["name", "stock_value"]),
    ("Stock Ledger Entry", "How many stock transfers occurred last month?", ["name", "posting_date"]),
    ("Item", "What is the stock aging report (items unsold for X days)?", ["name", "last_purchase_rate"]),
    ("Item", "List all items with negatives stock balance.", ["name", "allow_negative_stock"]),
    ("Stock Ledger Entry", "List all items with negatives stock balance.", ["name", "actual_qty"]),
    ("Stock Ledger Entry", "What is the total quantity of items received and issued this month?", ["name", "actual_qty"]),
    ("Sales Order", "What is the current status of sales orders (Pending, Delivered, Cancelled)?", ["status", "name"]),
    ("Sales Return", "How many sales returns were processed this quarter?", ["name"]),
    ("Project", "Which projects have exceeded their budget?", ["name", "total_costing_amount", "estimated_costing"]),
    ("Project", "What is the percentage completion of all active projects?", ["name", "percent_complete", "is_active", "monitor_progress"]),
    ("Task", "List overdue tasks assigned to a specific user.", ["name", "status", "owner", "exp_end_date"]),
    ("Project", "Which projects have resource shortages this month?", ["name", "total_costing_amount", "total_purchase_cost"]),
    ("Task", "How many new tasks were created last week?", ["creation"]),
    ("Task", "What is the average task completion time by project?", ["name", "project", "actual_time", "completed_on"]),
    ("Project", "List projects that have no tasks assigned.", ["name", "project_name"]),
    ("Task", "Show the workload distribution across project team members.", ["name", "owner", "project", "total_costing_amount"]),
    ("Sales Order", "What was the monthly trend of sales orders in the past year?", ["name", "transaction_date", "delivery_date"]),
    ("Sales Order", "Which sales orders have been delayed beyond their delivery dates?", ["name", "delivery_date", "status"]),
    ("Sales Invoice", "Show sales invoices that were fully paid within the last month.", ["name", "posting_date", "paid_amount", "status"]),
    ("Sales Order", "Which customers have the highest number of cancelled sales orders?", ["customer", "status", "name"]),
    ("Sales Order", "What is the breakdown of sales orders by region or territory?", ["territory", "customer", "name"]),
    ("Sales Invoice", "List the top 10 products by sales revenue this quarter.", ["items", "posting_date", "net_total"]),
    ("Sales Order", "How many sales orders were amended or updated last month?", ["name", "modified"]),
    ("Sales Order", "What is the average time between sales order creation and delivery?", ["transaction_date", "delivery_date", "name"]),
    ("Account", "What is the aging analysis of accounts receivable?", ["name", "account_type", "account_balance"]),
    ("Purchase Invoice", "List all unpaid vendor invoices by due date.", ["name", "due_date", "outstanding_amount"]),
    ("Account", "What is the total amount of bad debts written off last year?", ["name", "account_type", "balance_must_be"]),
    ("Journal Entry", "How many journal entries were reversed this quarter?", ["reversal_of", "posting_date", "company"]),
    ("Account", "What is the budget variance report for this fiscal year?", ["name", "report_type", "balance_must_be"]),
    ("Account", "Which accounts have the highest transaction volumes?", ["name", "account_type"]),
    ("Payment Entry", "List tax payments made in the last financial year.", ["payment_type", "posting_date", "taxes"]),
    ("Payment Entry", "What is the cash balance forecast for next month?", ["posting_date", "paid_amount", "received_amount"]),
    ("Purchase Order", "Which purchase orders are pending approval?", ["status", "name"]),
    ("Purchase Invoice", "What is the monthly purchase volume trend for key suppliers?", ["supplier", "posting_date"]),
    ("Purchase Invoice", "List all purchase invoices paid late in the last quarter.", ["posting_date", "status"]),
    ("Purchase Order", "How many purchase orders are currently in draft status?", ["docstatus", "status"]),
    ("Purchase Order", "What is the lead time variability for recent purchase orders?", ["transaction_date", "expected_delivery_date"]),
    ("Purchase Order", "Which suppliers have the best on-time delivery rates?", ["supplier", "delivery_date"]),
    ("Purchase Order", "List the purchase orders with the highest discounts applied.", ["discount_amount"]),
    ("Purchase Return", "How many purchase returns were processed last year?", ["name"]),
    ("Item", "What are the slow-moving items in the warehouse?", ["name", "shelf_life_in_days"]),
    ("Warehouse", "Which warehouses have the highest stock turnover rate?", ["name"]),
    ("Stock Ledger Entry", "Which warehouses have the highest stock turnover rate?", ["warehouse"]),
    ("Item", "Show items with expired batches or near expiry.", ["has_expiry_date"]),
    ("Stock Ledger Entry", "Show items with expired batches or near expiry.", ["batch_no"]),
    ("Stock Ledger Entry", "How many stock adjustments were made last month?", ["is_adjustment_entry", "name"]),
    ("Item", "What is the average replenishment time for critical items?", ["reorder_levels", "lead_time_days"]),
    ("Stock Ledger Entry", "List items with stock discrepancies between physical and system counts.", ["actual_qty"]),
    ("Item", "List items with stock discrepancies between physical and system counts.", ["allow_negative_stock"]),
    ("Warehouse", "What is the distribution of stock quantities across warehouses?", ["name"]),
    ("Stock Ledger Entry", "What is the distribution of stock quantities across warehouses?", ["warehouse"]),
    ("Item", "Which items have the highest valuation rate in inventory?", ["valuation_rate", "name"]),
    ("Purchase Invoice", "List all unpaid vendor bills older than 60 days.", ["name", "outstanding_amount"]),
    ("Stock Ledger Entry", "What is the turnover rate for key inventory items?", ["item_code", "posting_date", "actual_qty", "warehouse"]),
    ("Warehouse", "Which warehouses have the highest stock discrepancies?", ["name", "disabled"]),
    ("Item", "List items with expiry dates within next X days.", ["name", "has_expiry_date", "end_of_life"]),
    ("Stock Ledger Entry", "What is the total quantity of damaged stock?", ["actual_qty", "is_adjustment_entry", "posting_date"]),
    ("Stock Ledger Entry", "How many stock reconciliation entries were made last month?", ["posting_date", "is_adjustment_entry", "voucher_type"]),
    ("Item", "Which items have the highest holding cost?", ["name", "valuation_rate", "standard_rate"]),
    ("Stock Ledger Entry", "What is the average time stock remains in warehouse before issue?", ["posting_date", "warehouse", "actual_qty"]),
    ("Warehouse", "List warehouses with negatives stock balances.", ["name", "disabled"]),
    ("Purchase Invoice", "What is the average purchase order value?", ["total", "base_total"]),
    ("Purchase Invoice", "Which suppliers have the best on-time delivery performance?", ["supplier", "posting_date"]),
    ("Purchase Invoice", "How many purchase invoices are overdue for payment?", ["due_date", "status"]),
    ("Purchase Order", "List purchase orders pending receipt of goods.", ["status", "per_received"]),
    ("Purchase Invoice", "What is the total purchase return value this year?", ["is_return", "total"]),
    ("Item", "Which items have the longest supplier lead times?", ["lead_time_days", "supplier_items"]),
    ("Purchase Invoice", "What is the payment status of purchase invoices by supplier?", ["supplier", "is_paid"]),
    ("Purchase Order", "List purchase orders with partial delivery.", ["per_received", "status"]),
    ("Sales Order", "What is the average delivery time for sales orders?", ["transaction_date", "delivery_date"]),
    ("Sales Order", "How many sales orders are delayed beyond expected delivery date?", ["delivery_date", "status"]),
    ("Sales Order", "Which sales orders have the highest discount applied?", ["discount_amount", "additional_discount_percentage"]),
    ("Sales Order", "List all customers with pending sales orders.", ["customer", "status"]),
    ("Sales Invoice", "What is the monthly sales trend for the last year?", ["posting_date", "total"]),
    ("Sales Order", "Which sales team members generated the highest revenue last quarter?", ["sales_team", "total"]),
    ("Sales Invoice", "What is the total tax collected from sales invoices this year?", ["total_taxes_and_charges"]),
    ("Sales Order", "List all canceled sales orders and reasons for cancellation.", ["status", "remarks"])
]


for doctype, question, expected_fields in test_cases:
    full_question = f"Doctype: {doctype}\nQuestion: {question}"
    if doctype not in doctype_meta:
        print(f"Skipping test, missing meta fields for doctype: {doctype}")
        continue

    candidate_fields = [format_field(f) for f in doctype_meta[doctype]]

    question_emb = model.encode(full_question, convert_to_tensor=True)
    fields_emb = model.encode(candidate_fields, convert_to_tensor=True)

    scores = cos_sim(question_emb, fields_emb)[0]

    ranked_fields = sorted(zip(doctype_meta[doctype], scores), key=lambda x: x[1], reverse=True)

    print(f"\nQuestion: {question}")
    print("Top ranked fields:")
    for field, score in ranked_fields[:5]:
        print(f"  {field}: {score:.4f}")

    top_fields = [f for f, _ in ranked_fields[:5]]
    hits = [f for f in expected_fields if f in top_fields]

    print(f"Expected fields in top 5: {hits} / {expected_fields}")

In [ ]:
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.util import cos_sim

# Load your doctype meta
doctype_meta = {dt: fields_info["fields"] for dt, fields_info in meta.items()}

# Load two models
model1 = SentenceTransformer("/content/drive/MyDrive/Changai/S2/Model/Test1")
model2 = SentenceTransformer("hyrinmansoor/text2frappe-s2-sbert")  # replace with your 2nd model ID

def format_field(field):
    return f"{field}: field in the document"

# Your test cases
test_cases = [
    ("Sales Invoice", "How many sales invoices are there?", []),
    ("Sales Invoice", "What is the total amount invoiced?", ["grand_total"]),
    ("Sales Invoice", "How many invoices are paid?", ["status"]),
    ("Sales Invoice", "How many invoices are unpaid?", ["outstanding_amount"]),
    ("Sales Invoice", "What is the latest invoice date?", ["posting_date"]),
    ("Sales Invoice", "Who is the customer on the last invoice?", ["customer_name"]),
    ("Sales Invoice", "What is the total tax amount?", ["total_taxes_and_charges"]),
    ("Sales Invoice", "How many invoices were created today?", ["posting_date"]),
    ("Sales Invoice", "What is the average invoice amount?", ["grand_total"]),
    ("Sales Invoice", "How many invoices have discounts?", ["discount_amount"]),
    ("Supplier", "Which suppliers provide electronic components?", ["supplier_name", "supplier_group"]),
    ("Supplier", "Show suppliers with deliveries pending this week.", ["supplier_name"]),
    ("Supplier", "List suppliers with credit limits above ₹1,00,000.", ["supplier_name"]),
    ("Supplier", "How many purchase orders were made to supplier ABC Corp last month?", ["supplier_name"]),
    ("Supplier", "Show supplier contact person and phone number.", ["supplier_primary_contact", "mobile_no"]),
    ("Supplier", "List suppliers with overdue payments.", ["supplier_name"]),
    ("Supplier", "Which suppliers have been blacklisted?", ["supplier_name"]),
    ("Supplier", "Show suppliers associated with purchase invoices this quarter.", ["supplier_name"]),
    ("Supplier", "List suppliers based on their delivery performance.", ["supplier_name"]),
    ("Supplier", "Which suppliers have contracts expiring soon?", ["supplier_name"]),
    ("Supplier", "List all suppliers.", ["supplier_name"]),
    ("Supplier", "Show details of supplier SUP-001.", ["supplier_name"]),
    ("Supplier", "What is the contact information for supplier ABC Corp?", ["supplier_name", "supplier_primary_contact", "mobile_no", "email_id"]),
    ("Supplier", "List suppliers from India.", ["supplier_name", "country"]),
    ("Supplier", "Show supplier addresses.", ["supplier_name", "primary_address"]),
    ("Supplier", "How many suppliers are currently active?", ["supplier_name"]),
    ("Supplier", "List suppliers with pending purchase orders.", ["supplier_name"]),
    ("Supplier", "Show suppliers with the highest rating.", ["supplier_name"]),
    ("Supplier", "What are the payment terms for supplier XYZ Ltd?", ["supplier_name", "payment_terms"]),
    ("Supplier", "List suppliers by industry type.", ["supplier_name", "supplier_group"]),
    ("Employee", "Who are the employees in the sales department?", ["employee_name", "department"]),
    ("Employee", "What is the annual CTC for employee EMP-0012?", ["employee_name", "ctc"]),
    ("Employee", "Show the monthly salary for all employees.", ["employee_name", "ctc"]),
    ("Employee", "List employees with a salary greater than ₹50,000.", ["employee_name", "ctc"]),
    ("Employee", "What is the total salary expense for this month?", ["ctc"]),
    ("Employee", "Which employees have received a salary increase this year?", ["employee_name", "ctc"]),
    ("Employee", "Show the salary mode (e.g., bank transfer, cheque) for employees.", ["employee_name", "salary_mode"]),
    ("Employee", "List employees whose CTC is below ₹30,000.", ["employee_name", "ctc"]),
    ("Employee", "How many leave days has employee EMP-0005 taken this year?", ["employee_name"]),
    ("Employee", "Show leave balance for all employees.", ["employee_name"]),
    ("Employee", "Which employees are currently on leave?", ["employee_name"]),
    ("Employee", "List employees with pending leave approvals.", ["employee_name"]),
    ("Employee", "How many sick leave days were taken last month?", []),
    ("Employee", "Show leave type details for employee EMP-0010.", ["employee_name"]),
    ("Employee", "Which employees have exhausted their annual leave quota?", ["employee_name"]),
    ("Item", "List all items in stock.", ["item_name", "item_code"]),
    ("Item", "Show details of item ITEM-0001.", ["item_code", "item_name"]),
    ("Item", "What is the price of item ITEM-0050?", ["item_code", "standard_rate"]),
    ("Item", "How many units of ITEM-003 are available?", ["item_code"]),
    ("Item", "List items with low stock levels.", ["item_code", "item_name"]),
    ("Item", "Show items belonging to the Electronics category.", ["item_code", "item_name", "item_group"]),
    ("Item", "What is the warranty period for item ITEM-002?", ["item_code", "warranty_period"]),
    ("Item", "List all stock items.", ["item_code", "item_name", "is_stock_item"]),
    ("Item", "Show supplier details for item ITEM-007.", ["item_code", "supplier_items"]),
    ("Item", "Which items have a reorder level below 10?", ["item_code", "safety_stock"]),
    ("Sales Invoice", "What is the total invoiced amount for last month?", ["grand_total", "posting_date"]),
    ("Employee", "What is the sum of all employee salaries?", ["ctc"]),
    ("Sales Invoice", "Show all sales invoices with their status.", ["name", "status"]),
    ("Sales Invoice", "What is the status of sales invoice SI-0001?", ["name", "status"]),
    ("Sales Invoice", "What is the total invoiced amount for last month?", ["grand_total", "posting_date"]),
    ("Sales Invoice", "Show all invoices created by a specific sales team member.", ["name", "sales_team"]),
    ("Sales Invoice", "Which invoices have discounts applied?", ["name", "discount_amount"]),
    ("Sales Invoice", "Show invoices with payments pending.", ["name", "outstanding_amount"]),
    ("Employee", "Which employees joined in the last quarter?", ["employee_name", "date_of_joining"]),
    ("Employee", "Show contact details for employee EMP-0002.", ["employee_name", "personal_email", "cell_number", "employee_number"]),
    ("Employee", "List employees who are currently inactive.", ["employee_name", "status"]),
    ("Sales Order", "Show total quantity ordered this month.", ["total_qty", "transaction_date"]),
    ("Sales Order", "Which sales orders have been delivered?", ["name", "delivery_status"]),
    ("Sales Order", "Show all sales orders with discounts applied.", ["name", "discount_amount"]),
    ("Sales Order", "List sales orders pending billing.", ["name", "billing_status"]),
    ("Item", "List all stock items available.", ["item_code", "item_name", "is_stock_item"]),
    ("Item", "Which items are classified under Electronics group?", ["item_code", "item_group"]),
    ("Item", "Show items with low stock levels.", ["item_code", "item_name"]),
    ("Item", "List items supplied by vendor XYZ.", ["item_code", "supplier_items"]),
    ("Item", "What products are available in the Electronics category?", ["item_code", "item_name", "item_group"]),
    ("Item", "Show all stock currently available.", ["item_code", "item_name", "is_stock_item"]),
    ("Item", "List products with low inventory levels.", ["item_code", "item_name"]),
    ("Item", "Which products need to be reordered?", ["item_code", "item_name", "safety_stock"]),
    ("Item", "Show details for SKU 12345.", ["item_code", "item_name"]),
    ("Item", "What is the price of the blue widget?", ["item_name", "standard_rate"]),
    ("Item", "How many units are in stock for each product?", ["item_code", "item_name"]),
    ("Item", "List all products with a warranty period over 1 year.", ["item_code", "warranty_period"]),
    ("Item", "Show the supplier for the red chair.", ["item_name", "supplier_items"]),
    ("Item", "Which products belong to the 'Furniture' group?", ["item_code", "item_name", "item_group"]),
    ("Supplier", "Which suppliers provide electronic components?", ["supplier_name", "supplier_group"]),
    ("Supplier", "Show suppliers with deliveries pending this week.", ["supplier_name"]),
    ("Supplier", "List suppliers with credit limits above ₹1,00,000.", ["supplier_name"]),
    ("Supplier", "How many purchase orders were made to supplier ABC Corp last month?", ["supplier_name"]),
    ("Supplier", "Show supplier contact person and phone number.", ["supplier_name", "supplier_primary_contact", "mobile_no"]),
    ("Supplier", "List suppliers with overdue payments.", ["supplier_name"]),
    ("Supplier", "Which suppliers have been blacklisted?", ["supplier_name"]),
    ("Supplier", "Show suppliers associated with purchase invoices this quarter.", ["supplier_name"]),
    ("Supplier", "List suppliers based on their delivery performance.", ["supplier_name"]),
    ("Supplier", "Which suppliers have contracts expiring soon?", ["supplier_name"]),

]

# Initialize results tracking
results = {"model1": {"correct": 0, "wrong": 0}, "model2": {"correct": 0, "wrong": 0}}

for doctype, question, expected_fields in test_cases:
    full_question = f"Doctype: {doctype}\nQuestion: {question}"

    if doctype not in doctype_meta:
        print(f"Skipping test, missing meta fields for doctype: {doctype}")
        continue

    candidate_fields = [format_field(f) for f in doctype_meta[doctype]]

    # Encode with both models
    emb1_question = model1.encode(full_question, convert_to_tensor=True)
    emb1_fields = model1.encode(candidate_fields, convert_to_tensor=True)

    emb2_question = model2.encode(full_question, convert_to_tensor=True)
    emb2_fields = model2.encode(candidate_fields, convert_to_tensor=True)

    # Compute similarity
    scores1 = cos_sim(emb1_question, emb1_fields)[0]
    scores2 = cos_sim(emb2_question, emb2_fields)[0]

    # Rank fields
    ranked_fields1 = sorted(zip(doctype_meta[doctype], scores1), key=lambda x: x[1], reverse=True)
    ranked_fields2 = sorted(zip(doctype_meta[doctype], scores2), key=lambda x: x[1], reverse=True)

    top_fields1 = [f for f, _ in ranked_fields1[:5]]
    top_fields2 = [f for f, _ in ranked_fields2[:5]]

    # Evaluate hits
    hits1 = [f for f in expected_fields if f in top_fields1]
    hits2 = [f for f in expected_fields if f in top_fields2]

    results["model1"]["correct"] += len(hits1)
    results["model1"]["wrong"] += max(len(expected_fields) - len(hits1), 0)
    results["model2"]["correct"] += len(hits2)
    results["model2"]["wrong"] += max(len(expected_fields) - len(hits2), 0)

    # Print top-ranked fields for each model
    print(f"\nQuestion: {question}")
    print("Model 1 top fields:")
    for field, score in ranked_fields1[:5]:
        print(f"  {field}: {score:.4f}")
    print(f"Hits: {hits1} / Expected: {expected_fields}")

    print("Model 2 top fields:")
    for field, score in ranked_fields2[:5]:
        print(f"  {field}: {score:.4f}")
    print(f"Hits: {hits2} / Expected: {expected_fields}")

# Summary
print("\n=== Summary ===")
for model_name, res in results.items():
    total = res["correct"] + res["wrong"]
    accuracy = res["correct"] / total * 100 if total > 0 else 0
    print(f"{model_name}: Correct {res['correct']}, Wrong {res['wrong']}, Accuracy: {accuracy:.2f}%")
